In [1]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
import os

os.chdir('/content/MyDrive/MyDrive/github/data/GW matrix')

In [ ]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from tqdm import tqdm
from pandas.api.types import CategoricalDtype

# 1. 코드 간 유사도 구하기

In [ ]:
piano_chord = pd.read_excel('piano_chord_revised.xlsx')

piano_chord.drop(['chord' , 'index1' , 'index2' , 'index3' , 'index'] , axis = 1 , inplace = True)

piano_chord

,,major_chord,C,Db,D,Eb,E,F,Gb,G,Ab,A,Bb,B
0,Cb,Cb,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,Cb,Cbm,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,Cb,Cb7,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,Cb,Cbm7,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,Cb,Cbmaj7,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,B#,B#dim,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
353,B#,B#dim7,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
354,B#,B#m7b5,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
355,B#,B#+,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
for i in range(21) :

  piano_chord.iloc[i*17,1] = piano_chord.iloc[i*17,1][:-1]

## 1) 코드 간 유사도 계산 함수 정의

In [ ]:
def chord_similarity(chord_1 ,chord_2) : 

  con_1 = piano_chord['major_chord'] == chord_1

  con_2 = piano_chord['major_chord'] == chord_2

  idx_list_1 = []

  idx_list_2 = []

  for i in range(2,14) :

    target_1 = piano_chord.loc[con_1 , ].iloc[0,i]

    target_2 = piano_chord.loc[con_2 , ].iloc[0,i]

    if target_1 == 1 :

      idx_list_1.append(i-2)

    if target_2 == 1 :

      idx_list_2.append(i-2)

  idx_list_1 = set(idx_list_1)
  idx_list_2 = set(idx_list_2)

  similarity = len(idx_list_1.intersection(idx_list_2))/len(idx_list_1.union(idx_list_2))

  similarity = round(similarity , 3)

  return similarity

## 2) 코드 간 유사도 matrix 정의

In [ ]:
similarity_df = pd.DataFrame(np.repeat(0 , len(piano_chord['major_chord'])**2).reshape(len(piano_chord['major_chord']),len(piano_chord['major_chord'])))

similarity_df.columns = piano_chord['major_chord']

similarity_df.insert(0 , 'major_chord' , piano_chord['major_chord'])

similarity_df

major_chord,major_chord,Cb,Cbm,Cb7,Cbm7,Cbmaj7,CbmM7,Cb6,Cbm6,Cbadd2,...,B#m6,B#add2,B#madd2,B#sus4,B#sus2,B#dim,B#dim7,B#m7b5,B#+,B#7sus4
0,Cb,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Cbm,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cb7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Cbm7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cbmaj7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,B#dim,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
353,B#dim7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
354,B#m7b5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
355,B#+,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 3) 코드 간 유사도 계산

In [ ]:
for chord_1 in tqdm(piano_chord['major_chord']) :

  for chord_2 in piano_chord['major_chord'] :

    similarity_df.loc[similarity_df['major_chord'] == chord_1 , chord_2] = chord_similarity(chord_1 ,chord_2)

similarity_df

100%|██████████| 357/357 [17:17<00:00,  2.91s/it]


major_chord,major_chord,Cb,Cbm,Cb7,Cbm7,Cbmaj7,CbmM7,Cb6,Cbm6,Cbadd2,...,B#m6,B#add2,B#madd2,B#sus4,B#sus2,B#dim,B#dim7,B#m7b5,B#+,B#7sus4
0,Cb,1.00,0.500,0.750,0.400,0.750,0.400,0.750,0.400,0.750,...,0.167,0.000,0.167,0.000,0.000,0.500,0.400,0.400,0.000,0.000
1,Cbm,0.50,1.000,0.400,0.750,0.400,0.750,0.400,0.750,0.400,...,0.000,0.167,0.167,0.000,0.200,0.200,0.167,0.167,0.000,0.000
2,Cb7,0.75,0.400,1.000,0.600,0.600,0.333,0.600,0.333,0.600,...,0.333,0.000,0.143,0.000,0.000,0.400,0.600,0.333,0.000,0.000
3,Cbm7,0.40,0.750,0.600,1.000,0.333,0.600,0.333,0.600,0.333,...,0.143,0.143,0.143,0.000,0.167,0.167,0.333,0.143,0.000,0.000
4,Cbmaj7,0.75,0.400,0.600,0.333,1.000,0.600,0.600,0.333,0.600,...,0.143,0.000,0.143,0.000,0.000,0.400,0.333,0.600,0.000,0.143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,B#dim,0.50,0.200,0.400,0.167,0.400,0.167,0.400,0.167,0.400,...,0.400,0.167,0.400,0.200,0.200,1.000,0.750,0.750,0.200,0.167
353,B#dim7,0.40,0.167,0.600,0.333,0.333,0.143,0.333,0.143,0.333,...,0.600,0.143,0.333,0.167,0.167,0.750,1.000,0.600,0.167,0.143
354,B#m7b5,0.40,0.167,0.333,0.143,0.600,0.333,0.333,0.143,0.333,...,0.333,0.143,0.333,0.167,0.167,0.750,0.600,1.000,0.167,0.333
355,B#+,0.00,0.000,0.000,0.000,0.000,0.000,0.167,0.167,0.000,...,0.167,0.400,0.167,0.200,0.200,0.200,0.167,0.167,1.000,0.167


# 2. 변수 생성

## 1) 데이터 전처리

In [ ]:
data = pd.read_csv('commu_meta.csv')

data.drop('Unnamed: 0' , axis = 1 , inplace = True)

data

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid,8,120,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001
1,cmajor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm'...",mid_low,8,80,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002
2,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid_high,8,150,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003
3,cmajor,"[['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Gm'...",mid,8,110,cinematic,pad,choir,standard,4/4,45,46,train,commu00004
4,aminor,"[['Am', 'Am', 'Am', 'Am', 'Em', 'Em', 'Em', 'E...",mid_low,4,60,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11139,cmajor,"[['Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7',...",mid_high,8,110,cinematic,pad,synth_pad,standard,4/4,83,84,val,commu11140
11140,cmajor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",mid_low,8,110,cinematic,riff,acoustic_guitar,standard,4/4,51,55,val,commu11141
11141,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",very_low,8,145,cinematic,bass,string_ensemble,standard,4/4,109,112,val,commu11142
11142,aminor,"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...",low,8,140,cinematic,accompaniment,string_cello,standard,4/4,55,115,val,commu11143


In [ ]:
data_progressions = data['chord_progressions']

for i in range(data_progressions.shape[0]):
    
    data_progressions[i] = data_progressions[i].replace("'", "").replace(" ", "").replace("[", "").replace("]", "")
    
    data_progressions[i] = data_progressions[i].split(",")

data['chord_progressions'] = data_progressions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


## 2) chord_list

In [ ]:
chord_list = []

for chords in tqdm(data_progressions) : 

  target = []

  for chord in chords :

    if len(target) == 0 :

      target.append(chord)

    elif chord != target[-1] :

      target.append(chord)

  chord_list.append(target)

100%|██████████| 11144/11144 [00:00<00:00, 23835.44it/s]


In [ ]:
data.insert(2 , 'chord_list' , chord_list)

data

,audio_key,chord_progressions,chord_list,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, C, C, C, C, C...","[Am, C, G, Dm, Am, C, G, D]",mid,8,120,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001
1,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[C, Dm, G7, C, Am, F, G, C]",mid_low,8,80,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[Am, F, E+, E]",mid_high,8,150,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003
3,cmajor,"[C, C, C, C, C, C, C, C, Gm, Gm, Gm, Gm, Gm, G...","[C, Gm, C, Gm, Dm, A#, F, G]",mid,8,110,cinematic,pad,choir,standard,4/4,45,46,train,commu00004
4,aminor,"[Am, Am, Am, Am, Em, Em, Em, Em, F, F, F, F, F...","[Am, Em, F, Am, Em, F]",mid_low,4,60,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11139,cmajor,"[Fmaj7, Fmaj7, Fmaj7, Fmaj7, Fmaj7, Fmaj7, Fma...","[Fmaj7, Em7, Fmaj7, G7, C, Fmaj7, Em7, Am7, Dm...",mid_high,8,110,cinematic,pad,synth_pad,standard,4/4,83,84,val,commu11140
11140,cmajor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[Am, Fmaj7, Cmaj7, G]",mid_low,8,110,cinematic,riff,acoustic_guitar,standard,4/4,51,55,val,commu11141
11141,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[Am, F, Am7, F]",very_low,8,145,cinematic,bass,string_ensemble,standard,4/4,109,112,val,commu11142
11142,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[Am, Am7, Am, Am7]",low,8,140,cinematic,accompaniment,string_cello,standard,4/4,55,115,val,commu11143


## 3) 'soc' , 'eoc' 토큰 추가

In [ ]:
for chord_list in data['chord_list'] :

  chord_list.insert(0 , 'soc')

  chord_list.append('eoc')

data.head()

,audio_key,chord_progressions,chord_list,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, C, C, C, C, C...","[soc, Am, C, G, Dm, Am, C, G, D, eoc]",mid,8,120,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001
1,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, G7, C, Am, F, G, C, eoc]",mid_low,8,80,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]",mid_high,8,150,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003
3,cmajor,"[C, C, C, C, C, C, C, C, Gm, Gm, Gm, Gm, Gm, G...","[soc, C, Gm, C, Gm, Dm, A#, F, G, eoc]",mid,8,110,cinematic,pad,choir,standard,4/4,45,46,train,commu00004
4,aminor,"[Am, Am, Am, Am, Em, Em, Em, Em, F, F, F, F, F...","[soc, Am, Em, F, Am, Em, F, eoc]",mid_low,4,60,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005


## 4) bpm_group

In [ ]:
data.insert(6 , 'bpm_group' , np.where(data['bpm'] <= 60, 'very_slow' ,
                                       np.where(data['bpm'] <= 90 , 'slow' , 
                                                np.where(data['bpm'] <= 120 , 'fast' , 'very_fast'))))

data.head()

,audio_key,chord_progressions,chord_list,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, C, C, C, C, C...","[soc, Am, C, G, Dm, Am, C, G, D, eoc]",mid,8,120,fast,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001
1,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, G7, C, Am, F, G, C, eoc]",mid_low,8,80,slow,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]",mid_high,8,150,very_fast,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003
3,cmajor,"[C, C, C, C, C, C, C, C, Gm, Gm, Gm, Gm, Gm, G...","[soc, C, Gm, C, Gm, Dm, A#, F, G, eoc]",mid,8,110,fast,cinematic,pad,choir,standard,4/4,45,46,train,commu00004
4,aminor,"[Am, Am, Am, Am, Em, Em, Em, Em, F, F, F, F, F...","[soc, Am, Em, F, Am, Em, F, eoc]",mid_low,4,60,very_slow,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005


In [ ]:
bpm_ord = ['very_slow' , 'slow' , 'fast' , 'very_fast']

bpm_order = CategoricalDtype(categories = bpm_ord , ordered = True)

data['bpm_group'] = data['bpm_group'].astype(bpm_order)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11144 entries, 0 to 11143
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   audio_key           11144 non-null  object  
 1   chord_progressions  11144 non-null  object  
 2   chord_list          11144 non-null  object  
 3   pitch_range         11144 non-null  object  
 4   num_measures        11144 non-null  int64   
 5   bpm                 11144 non-null  int64   
 6   bpm_group           11144 non-null  category
 7   genre               11144 non-null  object  
 8   track_role          11144 non-null  object  
 9   inst                11144 non-null  object  
 10  sample_rhythm       11144 non-null  object  
 11  time_signature      11144 non-null  object  
 12  min_velocity        11144 non-null  int64   
 13  max_velocity        11144 non-null  int64   
 14  split_data          11144 non-null  object  
 15  id                  11144 non-null  

# 2. n = 2

## 1) BPM 구분
### a) very_slow ( ~ 60bpm)
### b) slow (61 ~ 90 bpm)
### c) fast (91 ~ 120 bpm)
### d) very fast(121 bpm ~ )

In [ ]:
data.head()

,audio_key,chord_progressions,chord_list,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, C, C, C, C, C...","[soc, Am, C, G, Dm, Am, C, G, D, eoc]",mid,8,120,fast,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001
1,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, G7, C, Am, F, G, C, eoc]",mid_low,8,80,slow,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]",mid_high,8,150,very_fast,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003
3,cmajor,"[C, C, C, C, C, C, C, C, Gm, Gm, Gm, Gm, Gm, G...","[soc, C, Gm, C, Gm, Dm, A#, F, G, eoc]",mid,8,110,fast,cinematic,pad,choir,standard,4/4,45,46,train,commu00004
4,aminor,"[Am, Am, Am, Am, Em, Em, Em, Em, F, F, F, F, F...","[soc, Am, Em, F, Am, Em, F, eoc]",mid_low,4,60,very_slow,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005


In [ ]:
very_slow = data[data['bpm_group'] == 'very_slow']
very_slow.reset_index(drop = True , inplace = True)

slow = data[data['bpm_group'] == 'slow']
slow.reset_index(drop = True , inplace = True)

fast = data[data['bpm_group'] == 'fast']
fast.reset_index(drop = True , inplace = True)

very_fast = data[data['bpm_group'] == 'very_fast']
very_fast.reset_index(drop = True , inplace = True)

print('매우 느리게 : ' , len(very_slow))
print('-'*30)
print('느리게 : ' , len(slow))
print('-'*30)
print('빠르게 : ' , len(fast))
print('-'*30)
print('매우 빠르게 : ' , len(very_fast))

매우 느리게 :  1943
------------------------------
느리게 :  4870
------------------------------
빠르게 :  2442
------------------------------
매우 빠르게 :  1889


## 2) BPM별 TF-IDF 구하기

### a) Very_slow (bpm <= 60)

#### ㄱ) Chord Progression Token 구하기

In [ ]:
very_slow.head()

,audio_key,chord_progressions,chord_list,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Em, Em, Em, Em, F, F, F, F, F...","[soc, Am, Em, F, Am, Em, F, eoc]",mid_low,4,60,very_slow,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005
1,aminor,"[Am, Am, Am, Am, Em, Em, Em, Em, F, F, F, F, F...","[soc, Am, Em, F, Am, Em, F, eoc]",mid_high,4,60,very_slow,cinematic,main_melody,string_ensemble,standard,4/4,21,22,train,commu00011
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Dm, Dm, Dm, D...","[soc, Am, Dm, E, Am, eoc]",mid_high,4,60,very_slow,cinematic,main_melody,string_ensemble,standard,4/4,41,42,train,commu00014
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Dm, Dm, Dm, D...","[soc, Am, Dm, E, Am, eoc]",mid_low,4,60,very_slow,cinematic,pad,string_ensemble,standard,4/4,22,24,train,commu00015
4,cmajor,"[C, C, C, C, C, C, C, C, Dm7, Dm7, Dm7, Dm7, D...","[soc, C, Dm7, C, A#, C, Dm7, C, A#, eoc]",mid,8,50,very_slow,newage,pad,brass_ensemble,standard,4/4,81,82,train,commu00020


In [ ]:
total_chord_prog_n2 = []

for i in tqdm(range(len(very_slow))) : 

  target_list = very_slow.loc[i , 'chord_list'] 

  chord_prog_list = []

  for j in range(len(target_list)-1) :

    chord_prog = []

    chord_prog.append(target_list[j])
    chord_prog.append(target_list[j+1])

    chord_prog = tuple(chord_prog)

    chord_prog_list.append(chord_prog)

  total_chord_prog_n2.append(chord_prog_list)

total_chord_prog_n2

In [ ]:
very_slow.insert(3 , 'chord_prog_token_n2' , total_chord_prog_n2)

very_slow.head()

,audio_key,chord_progressions,chord_list,chord_prog_token,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Em, Em, Em, Em, F, F, F, F, F...","[soc, Am, Em, F, Am, Em, F, eoc]","[(soc, Am), (Am, Em), (Em, F), (F, Am), (Am, E...",mid_low,4,60,very_slow,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005
1,aminor,"[Am, Am, Am, Am, Em, Em, Em, Em, F, F, F, F, F...","[soc, Am, Em, F, Am, Em, F, eoc]","[(soc, Am), (Am, Em), (Em, F), (F, Am), (Am, E...",mid_high,4,60,very_slow,cinematic,main_melody,string_ensemble,standard,4/4,21,22,train,commu00011
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Dm, Dm, Dm, D...","[soc, Am, Dm, E, Am, eoc]","[(soc, Am), (Am, Dm), (Dm, E), (E, Am), (Am, e...",mid_high,4,60,very_slow,cinematic,main_melody,string_ensemble,standard,4/4,41,42,train,commu00014
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Dm, Dm, Dm, D...","[soc, Am, Dm, E, Am, eoc]","[(soc, Am), (Am, Dm), (Dm, E), (E, Am), (Am, e...",mid_low,4,60,very_slow,cinematic,pad,string_ensemble,standard,4/4,22,24,train,commu00015
4,cmajor,"[C, C, C, C, C, C, C, C, Dm7, Dm7, Dm7, Dm7, D...","[soc, C, Dm7, C, A#, C, Dm7, C, A#, eoc]","[(soc, C), (C, Dm7), (Dm7, C), (C, A#), (A#, C...",mid,8,50,very_slow,newage,pad,brass_ensemble,standard,4/4,81,82,train,commu00020


#### ㄴ) TF-IDF 구하기

In [ ]:
prog_token_n2 = []

for i in tqdm(range(len(very_slow))) :

  for chord_token in very_slow.loc[i , 'chord_prog_token_n2'] :

    if chord_token not in prog_token_n2 :

      prog_token_n2.append(chord_token)

100%|██████████| 1943/1943 [00:00<00:00, 32056.76it/s]


In [ ]:
very_slow_tf_idf_df_n2 = pd.DataFrame({'chord_prog_token_n2' : prog_token_n2})

very_slow_tf_idf_df_n2

,chord_prog_token
0,"(soc, Am)"
1,"(Am, Em)"
2,"(Em, F)"
3,"(F, Am)"
4,"(F, eoc)"
...,...
179,"(D#, G#maj7)"
180,"(G#maj7, eoc)"
181,"(G, Am7)"
182,"(Em7, Dm7)"


In [ ]:
very_slow_tf_idf_df_n2 = pd.concat([very_slow_tf_idf_df_n2 , pd.DataFrame(np.repeat(0 , len(very_slow_tf_idf_df_n2)*len(very_slow)).reshape(len(very_slow_tf_idf_df_n2) , len(very_slow)))] , axis = 1)

very_slow_tf_idf_df_n2.insert(len(very_slow) + 1 , 'IDF-Count' , np.repeat(0 , len(very_slow_tf_idf_df_n2)))

very_slow_tf_idf_df_n2                                                      

,chord_prog_token,0,1,2,3,4,5,6,7,8,...,1934,1935,1936,1937,1938,1939,1940,1941,1942,IDF-Count
0,"(soc, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(Am, Em)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(Em, F)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(F, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(F, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,"(D#, G#maj7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180,"(G#maj7, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
181,"(G, Am7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
182,"(Em7, Dm7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for chord_prog_token in tqdm(prog_token_n2) :

  idf_count = 0

  for i in range(len(very_slow)) : 

    target_list = very_slow.loc[i , 'chord_prog_token_n2']

    if chord_prog_token not in target_list :

      tf_score = 0

    else :

      idf_count += 1

      each_chord_prog_token_len = len(target_list)

      count = target_list.count(chord_prog_token)

      tf_score = count/each_chord_prog_token_len

      tf_score = round(tf_score , 4)

    very_slow_tf_idf_df_n2.loc[very_slow_tf_idf_df_n2['chord_prog_token_n2'] == chord_prog_token , i] = tf_score

  very_slow_tf_idf_df_n2.loc[very_slow_tf_idf_df_n2['chord_prog_token_n2'] == chord_prog_token , 'IDF-Count'] = idf_count

100%|██████████| 184/184 [03:28<00:00,  1.13s/it]


In [ ]:
very_slow_tf_idf_df_n2

,chord_prog_token,0,1,2,3,4,5,6,7,8,...,1934,1935,1936,1937,1938,1939,1940,1941,1942,IDF-Count
0,"(soc, Am)",0.1429,0.1429,0.2,0.2,0.0,0.0,0.2,0.1111,0.000,...,0.0,0.0,0.0,0.0,0.5,0.000,0.000,0.0000,0.0833,381
1,"(Am, Em)",0.2857,0.2857,0.0,0.0,0.0,0.0,0.0,0.1111,0.000,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0000,0.0000,174
2,"(Em, F)",0.2857,0.2857,0.0,0.0,0.0,0.0,0.0,0.0000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0000,0.0000,312
3,"(F, Am)",0.1429,0.1429,0.0,0.0,0.0,0.0,0.0,0.1111,0.125,...,0.0,0.0,0.0,0.0,0.0,0.125,0.125,0.0000,0.0000,301
4,"(F, eoc)",0.1429,0.1429,0.0,0.0,0.0,0.0,0.0,0.1111,0.125,...,0.0,0.0,0.0,0.0,0.0,0.125,0.125,0.0000,0.0000,569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,"(D#, G#maj7)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0000,0.0000,1
180,"(G#maj7, eoc)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0000,0.0000,1
181,"(G, Am7)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.1111,0.0000,4
182,"(Em7, Dm7)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0000,0.0000,2


In [ ]:
import math

idf_score_list_n2 = []

for i in tqdm(range(len(very_slow_tf_idf_df_n2))) : 

  idf_score = math.log(len(very_slow)/(1 + very_slow_tf_idf_df_n2.loc[i , 'IDF-Count']))

  idf_score_list_n2.append(idf_score)

very_slow_tf_idf_df_n2.insert(len(very_slow) + 2 , 'IDF-Score' , idf_score_list_n2)

very_slow_tf_idf_df_n2

100%|██████████| 184/184 [00:00<00:00, 21784.29it/s]
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


,chord_prog_token,0,1,2,3,4,5,6,7,8,...,1935,1936,1937,1938,1939,1940,1941,1942,IDF-Count,IDF-Score
0,"(soc, Am)",0.1429,0.1429,0.2,0.2,0.0,0.0,0.2,0.1111,0.000,...,0.0,0.0,0.0,0.5,0.000,0.000,0.0000,0.0833,381,1.626568
1,"(Am, Em)",0.2857,0.2857,0.0,0.0,0.0,0.0,0.0,0.1111,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.0000,0.0000,174,2.407202
2,"(Em, F)",0.2857,0.2857,0.0,0.0,0.0,0.0,0.0,0.0000,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.0000,0.0000,312,1.825785
3,"(F, Am)",0.1429,0.1429,0.0,0.0,0.0,0.0,0.0,0.1111,0.125,...,0.0,0.0,0.0,0.0,0.125,0.125,0.0000,0.0000,301,1.861561
4,"(F, eoc)",0.1429,0.1429,0.0,0.0,0.0,0.0,0.0,0.1111,0.125,...,0.0,0.0,0.0,0.0,0.125,0.125,0.0000,0.0000,569,1.226352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,"(D#, G#maj7)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.0000,0.0000,1,6.878841
180,"(G#maj7, eoc)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.0000,0.0000,1,6.878841
181,"(G, Am7)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.1111,0.0000,4,5.962551
182,"(Em7, Dm7)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.0000,0.0000,2,6.473376


In [ ]:
very_slow_tf_idf_df_n2.to_csv('TF-IDF_very_slow_n2.csv')

#### ㄷ) TF-IDF Bag of Chord 만들기

In [ ]:
very_slow_tf_idf_score_n2 = pd.DataFrame({'chord_prog_token_n2' : prog_token_n2})

very_slow_tf_idf_score_n2 = pd.concat([very_slow_tf_idf_score_n2 , pd.DataFrame(np.repeat(0 , len(very_slow_tf_idf_score_n2)*len(very_slow)).reshape(len(very_slow_tf_idf_score_n2) , len(very_slow)))] , axis = 1)
                       
very_slow_tf_idf_score_n2

,chord_prog_token,0,1,2,3,4,5,6,7,8,...,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942
0,"(soc, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(Am, Em)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(Em, F)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(F, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(F, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,"(D#, G#maj7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180,"(G#maj7, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
181,"(G, Am7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
182,"(Em7, Dm7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in tqdm(range(1 , len(very_slow) + 1)) :

  very_slow_tf_idf_score_n2.iloc[: , i] = very_slow_tf_idf_df_n2.iloc[:,i]*very_slow_tf_idf_df_n2.loc[:,'IDF-Score']

very_slow_tf_idf_score_n2

100%|██████████| 1943/1943 [00:01<00:00, 1019.09it/s]


,chord_prog_token,0,1,2,3,4,5,6,7,8,...,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942
0,"(soc, Am)",0.232437,0.232437,0.325314,0.325314,0.0,0.0,0.325314,0.180712,0.000000,...,0.095642,0.0,0.0,0.0,0.0,0.813284,0.000000,0.000000,0.000000,0.135493
1,"(Am, Em)",0.687738,0.687738,0.000000,0.000000,0.0,0.0,0.000000,0.267440,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,"(Em, F)",0.521627,0.521627,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,"(F, Am)",0.266017,0.266017,0.000000,0.000000,0.0,0.0,0.000000,0.206819,0.232695,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.232695,0.232695,0.000000,0.000000
4,"(F, eoc)",0.175246,0.175246,0.000000,0.000000,0.0,0.0,0.000000,0.136248,0.153294,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.153294,0.153294,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,"(D#, G#maj7)",0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
180,"(G#maj7, eoc)",0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
181,"(G, Am7)",0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.662439,0.000000
182,"(Em7, Dm7)",0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
very_slow_tf_idf_boc_n2 = pd.DataFrame(very_slow_tf_idf_score_n2.T)

very_slow_tf_idf_boc_n2

,0,1,2,3,4,5,6,7,8,9,...,174,175,176,177,178,179,180,181,182,183
chord_prog_token,"(soc, Am)","(Am, Em)","(Em, F)","(F, Am)","(F, eoc)","(Am, Dm)","(Dm, E)","(E, Am)","(Am, eoc)","(soc, C)",...,"(soc, Cm)","(Cm, D#)","(D#, Cm)","(Cm, C#)","(C#, Cm)","(D#, G#maj7)","(G#maj7, eoc)","(G, Am7)","(Em7, Dm7)","(G, Dm)"
0,0.232437,0.687738,0.521627,0.266017,0.175246,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.232437,0.687738,0.521627,0.266017,0.175246,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.325314,0.0,0.0,0.0,0.0,0.692223,0.692223,0.482587,0.365797,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.325314,0.0,0.0,0.0,0.0,0.692223,0.692223,0.482587,0.365797,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938,0.813284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.914493,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1939,0.0,0.0,0.0,0.232695,0.153294,0.0,0.0,0.0,0.0,0.087872,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1940,0.0,0.0,0.0,0.232695,0.153294,0.0,0.0,0.0,0.0,0.087872,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.662439,0.0,0.0


In [ ]:
tf_idf_boc_list_n2 = []

for i in tqdm(range(1, len(very_slow_tf_idf_boc_n2))) : 

  tf_idf_boc_list_n2.append(list(very_slow_tf_idf_boc_n2.iloc[i , :]))

100%|██████████| 1943/1943 [00:00<00:00, 7886.92it/s]


In [ ]:
very_slow_tf_idf_boc_n2 = pd.DataFrame({'id' : very_slow['id']})

very_slow_tf_idf_boc_n2.insert(1 , 'TF-IDF Bag of Chords' , tf_idf_boc_list_n2)

very_slow_tf_idf_boc_n2

,id,TF-IDF Bag of Chords
0,commu00005,"[0.23243654444615658, 0.6877377472371866, 0.52..."
1,commu00011,"[0.23243654444615658, 0.6877377472371866, 0.52..."
2,commu00014,"[0.325313568154173, 0.0, 0.0, 0.0, 0.0, 0.6922..."
3,commu00015,"[0.325313568154173, 0.0, 0.0, 0.0, 0.0, 0.6922..."
4,commu00020,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
1938,commu10843,"[0.8132839203854324, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1939,commu10883,"[0.0, 0.0, 0.0, 0.2326951790003213, 0.15329401..."
1940,commu10899,"[0.0, 0.0, 0.0, 0.2326951790003213, 0.15329401..."
1941,commu11110,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
very_slow_tf_idf_boc_n2.to_csv('TF-IDF_Bag_of_Chords_very_slow_n2.csv')

### b) Slow (61 < bpm <= 90)

#### ㄱ) Chord Progression Token 구하기

In [ ]:
slow.head()

,audio_key,chord_progressions,chord_list,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,TF-IDF Bag of Chords
0,cmajor,"[Fmaj7, Fmaj7, Fmaj7, Fmaj7, Am7, Am7, Am7, Am...","[soc, Fmaj7, Am7, G, Cmaj7, eoc]",mid_high,4.0,65,newage,main_melody,acoustic_piano,standard,4/4,46,61,train,commu00043,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,cmajor,"[Fmaj7, Fmaj7, Fmaj7, Fmaj7, Am7, Am7, Am7, Am...","[soc, Fmaj7, Am7, G, Cmaj7, eoc]",high,4.0,65,newage,sub_melody,acoustic_piano,standard,4/4,63,66,train,commu00048,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,cmajor,"[Fmaj7, Fmaj7, Fmaj7, Fmaj7, Am7, Am7, Am7, Am...","[soc, Fmaj7, Am7, G, Cmaj7, eoc]",mid_low,4.0,65,newage,accompaniment,acoustic_piano,standard,4/4,43,43,train,commu00054,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Gadd2, Gadd2,...","[soc, Am, Gadd2, Fmaj7, G, Cmaj7, Dm7, Bbmaj7,...",mid_high,8.0,80,cinematic,main_melody,string_ensemble,standard,4/4,90,90,train,commu00057,"[0.1278784109036946, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,cmajor,"[Fmaj7, Fmaj7, Fmaj7, Fmaj7, Am7, Am7, Am7, Am...","[soc, Fmaj7, Am7, G, Cmaj7, eoc]",mid_low,4.0,65,newage,accompaniment,acoustic_piano,standard,4/4,80,118,train,commu00059,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
total_chord_prog_n2 = []

for i in tqdm(range(len(slow))) : 

  target_list = slow.loc[i , 'chord_list'] 

  chord_prog_list = []

  for j in range(len(target_list)-1) :

    chord_prog = []

    chord_prog.append(target_list[j])
    chord_prog.append(target_list[j+1])

    chord_prog = tuple(chord_prog)

    chord_prog_list.append(chord_prog)

  total_chord_prog_n2.append(chord_prog_list)

total_chord_prog_n2

In [ ]:
slow.insert(3 , 'chord_prog_token_n2' , total_chord_prog_n2)

slow.head()

,audio_key,chord_progressions,chord_list,chord_prog_token_n2,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, G7, C, Am, F, G, C, eoc]","[(soc, C), (C, Dm), (Dm, G7), (G7, C), (C, Am)...",mid_low,8,80,slow,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002
1,cmajor,"[Fmaj7, Fmaj7, Fmaj7, Fmaj7, Fmaj7, Fmaj7, Fma...","[soc, Fmaj7, Em7, Dm7, Cmaj7, eoc]","[(soc, Fmaj7), (Fmaj7, Em7), (Em7, Dm7), (Dm7,...",mid,4,65,slow,newage,pad,string_ensemble,standard,4/4,53,60,train,commu00023
2,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, G7, C, Am, F, G, C, eoc]","[(soc, C), (C, Dm), (Dm, G7), (G7, C), (C, Am)...",mid_high,8,80,slow,newage,main_melody,acoustic_piano,standard,4/4,23,30,train,commu00024
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Em, Em, Em, E...","[soc, Am, Em, F, Em, Am, Em, F, E, eoc]","[(soc, Am), (Am, Em), (Em, F), (F, Em), (Em, A...",mid_low,8,70,slow,newage,accompaniment,acoustic_piano,standard,4/4,21,24,train,commu00036
4,cmajor,"[C, C, C, C, C, C, C, C, F, F, F, F, G, G, G, ...","[soc, C, F, G, C, F, Em, C, F, G, C, F, Em, eoc]","[(soc, C), (C, F), (F, G), (G, C), (C, F), (F,...",mid_high,8,80,slow,newage,main_melody,acoustic_piano,standard,4/4,21,22,train,commu00041


#### ㄴ) TF-IDF 구하기

In [ ]:
prog_token_n2 = []

for i in tqdm(range(len(slow))) :

  for chord_token in slow.loc[i , 'chord_prog_token_n2'] :

    if chord_token not in prog_token_n2 :

      prog_token_n2.append(chord_token)

100%|██████████| 4870/4870 [00:00<00:00, 26264.66it/s]


In [ ]:
slow_tf_idf_df_n2 = pd.DataFrame({'chord_prog_token_n2' : prog_token_n2})

slow_tf_idf_df_n2

,chord_prog_token_n2
0,"(soc, C)"
1,"(C, Dm)"
2,"(Dm, G7)"
3,"(G7, C)"
4,"(C, Am)"
...,...
242,"(Cm, Gm)"
243,"(Gm, G#maj7)"
244,"(G#maj7, Cm)"
245,"(G#maj7, eoc)"


In [ ]:
slow_tf_idf_df_n2 = pd.concat([slow_tf_idf_df_n2 , pd.DataFrame(np.repeat(0 , len(slow_tf_idf_df_n2)*len(slow)).reshape(len(slow_tf_idf_df_n2) , len(slow)))] , axis = 1)

slow_tf_idf_df_n2.insert(len(slow) + 1 , 'IDF-Count' , np.repeat(0 , len(slow_tf_idf_df_n2)))

slow_tf_idf_df_n2

,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,4861,4862,4863,4864,4865,4866,4867,4868,4869,IDF-Count
0,"(soc, C)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(C, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(Dm, G7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(G7, C)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(C, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,"(Cm, Gm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243,"(Gm, G#maj7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
244,"(G#maj7, Cm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
245,"(G#maj7, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for chord_prog_token in tqdm(prog_token_n2) :

  idf_count = 0

  for i in range(len(slow)) : 

    target_list = slow.loc[i , 'chord_prog_token_n2']

    if chord_prog_token not in target_list :

      tf_score = 0

    else :

      idf_count += 1

      each_chord_prog_token_len = len(target_list)

      count = target_list.count(chord_prog_token)

      tf_score = count/each_chord_prog_token_len

      tf_score = round(tf_score , 4)

    slow_tf_idf_df_n2.loc[slow_tf_idf_df_n2['chord_prog_token_n2'] == chord_prog_token , i] = tf_score

  slow_tf_idf_df_n2.loc[slow_tf_idf_df_n2['chord_prog_token_n2'] == chord_prog_token , 'IDF-Count'] = idf_count

100%|██████████| 247/247 [13:47<00:00,  3.35s/it]


In [ ]:
slow_tf_idf_df_n2

,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,4861,4862,4863,4864,4865,4866,4867,4868,4869,IDF-Count
0,"(soc, C)",0.1111,0.0,0.1111,0.0,0.0769,0.0,0.1111,0.1111,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1046
1,"(C, Dm)",0.1111,0.0,0.1111,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118
2,"(Dm, G7)",0.1111,0.0,0.1111,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,"(G7, C)",0.1111,0.0,0.1111,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,177
4,"(C, Am)",0.1111,0.0,0.1111,0.0,0.0000,0.0,0.0000,0.2222,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,"(Cm, Gm)",0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
243,"(Gm, G#maj7)",0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
244,"(G#maj7, Cm)",0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
245,"(G#maj7, eoc)",0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [ ]:
import math

idf_score_list_n2 = []

for i in tqdm(range(len(slow_tf_idf_df_n2))) : 

  idf_score = math.log(len(slow)/(1 + slow_tf_idf_df_n2.loc[i , 'IDF-Count']))

  idf_score_list_n2.append(idf_score)

slow_tf_idf_df_n2.insert(len(slow) + 2 , 'IDF-Score' , idf_score_list_n2)

slow_tf_idf_df_n2

100%|██████████| 247/247 [00:00<00:00, 30694.27it/s]
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,4862,4863,4864,4865,4866,4867,4868,4869,IDF-Count,IDF-Score
0,"(soc, C)",0.1111,0.0,0.1111,0.0,0.0769,0.0,0.1111,0.1111,0.0,...,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1046,1.537165
1,"(C, Dm)",0.1111,0.0,0.1111,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118,3.711726
2,"(Dm, G7)",0.1111,0.0,0.1111,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,6.699090
3,"(G7, C)",0.1111,0.0,0.1111,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,177,3.309066
4,"(C, Am)",0.1111,0.0,0.1111,0.0,0.0000,0.0,0.0000,0.2222,0.0,...,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,234,3.031264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,"(Cm, Gm)",0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,7.392237
243,"(Gm, G#maj7)",0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,7.392237
244,"(G#maj7, Cm)",0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,7.392237
245,"(G#maj7, eoc)",0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,7.392237


In [ ]:
slow_tf_idf_df_n2.to_csv('TF-IDF_slow_n2.csv')

#### ㄷ) TF-IDF Bag of Chord 만들기

In [ ]:
slow_tf_idf_score_n2 = pd.DataFrame({'chord_prog_token_n2' : prog_token_n2})

slow_tf_idf_score_n2 = pd.concat([slow_tf_idf_score_n2 , pd.DataFrame(np.repeat(0 , len(slow_tf_idf_score_n2)*len(slow)).reshape(len(slow_tf_idf_score_n2) , len(slow)))] , axis = 1)
                       
slow_tf_idf_score_n2

,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,4860,4861,4862,4863,4864,4865,4866,4867,4868,4869
0,"(soc, C)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(C, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(Dm, G7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(G7, C)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(C, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,"(Cm, Gm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243,"(Gm, G#maj7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
244,"(G#maj7, Cm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
245,"(G#maj7, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in tqdm(range(1 , len(slow) + 1)) :

  slow_tf_idf_score_n2.iloc[: , i] = slow_tf_idf_df_n2.iloc[:,i]*slow_tf_idf_df_n2.loc[:,'IDF-Score']

slow_tf_idf_score_n2

100%|██████████| 4870/4870 [00:10<00:00, 486.80it/s] 


,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,4860,4861,4862,4863,4864,4865,4866,4867,4868,4869
0,"(soc, C)",0.170779,0.0,0.170779,0.0,0.118208,0.0,0.170779,0.170779,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"(C, Dm)",0.412373,0.0,0.412373,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"(Dm, G7)",0.744269,0.0,0.744269,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"(G7, C)",0.367637,0.0,0.367637,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"(C, Am)",0.336773,0.0,0.336773,0.0,0.000000,0.0,0.000000,0.673547,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,"(Cm, Gm)",0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
243,"(Gm, G#maj7)",0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
244,"(G#maj7, Cm)",0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
245,"(G#maj7, eoc)",0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
slow_tf_idf_boc_n2 = pd.DataFrame(slow_tf_idf_score_n2.T)

slow_tf_idf_boc_n2

,0,1,2,3,4,5,6,7,8,9,...,237,238,239,240,241,242,243,244,245,246
chord_prog_token_n2,"(soc, C)","(C, Dm)","(Dm, G7)","(G7, C)","(C, Am)","(Am, F)","(F, G)","(G, C)","(C, eoc)","(soc, Fmaj7)",...,"(Am, Cmaj7)","(Em7, G)","(Am7, Dm)","(Dm, eoc)","(soc, Cm)","(Cm, Gm)","(Gm, G#maj7)","(G#maj7, Cm)","(G#maj7, eoc)","(Am, Dm)"
0,0.170779,0.412373,0.744269,0.367637,0.336773,0.343597,0.279643,0.287248,0.252005,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.414497,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.170779,0.412373,0.744269,0.367637,0.336773,0.343597,0.279643,0.287248,0.252005,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4866,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tf_idf_boc_list_n2 = []

for i in tqdm(range(1, len(slow_tf_idf_boc_n2))) : 

  tf_idf_boc_list_n2.append(list(slow_tf_idf_boc_n2.iloc[i , :]))

100%|██████████| 4870/4870 [00:00<00:00, 7494.44it/s]


In [ ]:
slow_tf_idf_boc_n2 = pd.DataFrame({'id' : slow['id']})

slow_tf_idf_boc_n2.insert(1 , 'TF-IDF Bag of Chords' , tf_idf_boc_list_n2)

slow_tf_idf_boc_n2

,id,TF-IDF Bag of Chords
0,commu00002,"[0.17077903207839756, 0.4123727278214233, 0.74..."
1,commu00023,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,commu00024,"[0.17077903207839756, 0.4123727278214233, 0.74..."
3,commu00036,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,commu00041,"[0.11820798890034898, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...
4865,commu11125,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4866,commu11128,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4867,commu11137,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4868,commu11138,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
slow_tf_idf_boc_n2.to_csv('TF-IDF_Bag_of_Chords_slow_n2.csv')

### C) Fast (90 < bpm <= 120)

#### ㄱ) Chord Progression Token 구하기

In [ ]:
fast.head()

,audio_key,chord_progressions,chord_list,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, C, C, C, C, C...","[soc, Am, C, G, Dm, Am, C, G, D, eoc]",mid,8,120,fast,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001
1,cmajor,"[C, C, C, C, C, C, C, C, Gm, Gm, Gm, Gm, Gm, G...","[soc, C, Gm, C, Gm, Dm, A#, F, G, eoc]",mid,8,110,fast,cinematic,pad,choir,standard,4/4,45,46,train,commu00004
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, C, C, C, C, C...","[soc, Am, C, G, D, Am, C, G, D, eoc]",mid_high,8,120,fast,cinematic,riff,string_ensemble,standard,4/4,121,122,train,commu00006
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, D, D, D, D, D...","[soc, Am, D, Am, D, Am, D, E, eoc]",mid_low,8,110,fast,cinematic,accompaniment,string_ensemble,standard,4/4,87,96,train,commu00007
4,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, Am, Gsus4, eoc]",mid,4,120,fast,cinematic,sub_melody,brass_ensemble,standard,4/4,68,92,train,commu00008


In [ ]:
total_chord_prog_n2 = []

for i in tqdm(range(len(fast))) : 

  target_list = fast.loc[i , 'chord_list'] 

  chord_prog_list = []

  for j in range(len(target_list)-1) :

    chord_prog = []

    chord_prog.append(target_list[j])
    chord_prog.append(target_list[j+1])

    chord_prog = tuple(chord_prog)

    chord_prog_list.append(chord_prog)

  total_chord_prog_n2.append(chord_prog_list)

total_chord_prog_n2

In [ ]:
fast.insert(3 , 'chord_prog_token_n2' , total_chord_prog_n2)

fast.head()

,audio_key,chord_progressions,chord_list,chord_prog_token_n2,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, C, C, C, C, C...","[soc, Am, C, G, Dm, Am, C, G, D, eoc]","[(soc, Am), (Am, C), (C, G), (G, Dm), (Dm, Am)...",mid,8,120,fast,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001
1,cmajor,"[C, C, C, C, C, C, C, C, Gm, Gm, Gm, Gm, Gm, G...","[soc, C, Gm, C, Gm, Dm, A#, F, G, eoc]","[(soc, C), (C, Gm), (Gm, C), (C, Gm), (Gm, Dm)...",mid,8,110,fast,cinematic,pad,choir,standard,4/4,45,46,train,commu00004
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, C, C, C, C, C...","[soc, Am, C, G, D, Am, C, G, D, eoc]","[(soc, Am), (Am, C), (C, G), (G, D), (D, Am), ...",mid_high,8,120,fast,cinematic,riff,string_ensemble,standard,4/4,121,122,train,commu00006
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, D, D, D, D, D...","[soc, Am, D, Am, D, Am, D, E, eoc]","[(soc, Am), (Am, D), (D, Am), (Am, D), (D, Am)...",mid_low,8,110,fast,cinematic,accompaniment,string_ensemble,standard,4/4,87,96,train,commu00007
4,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, Am, Gsus4, eoc]","[(soc, C), (C, Dm), (Dm, Am), (Am, Gsus4), (Gs...",mid,4,120,fast,cinematic,sub_melody,brass_ensemble,standard,4/4,68,92,train,commu00008


#### ㄴ) TF-IDF 구하기

In [ ]:
prog_token_n2 = []

for i in tqdm(range(len(fast))) :

  for chord_token in fast.loc[i , 'chord_prog_token_n2'] :

    if chord_token not in prog_token_n2 :

      prog_token_n2.append(chord_token)

100%|██████████| 2442/2442 [00:00<00:00, 5108.33it/s]


In [ ]:
fast_tf_idf_df_n2 = pd.DataFrame({'chord_prog_token_n2' : prog_token_n2})

fast_tf_idf_df_n2

,chord_prog_token_n2
0,"(soc, Am)"
1,"(Am, C)"
2,"(C, G)"
3,"(G, Dm)"
4,"(Dm, Am)"
...,...
152,"(Fmaj7, eoc)"
153,"(Fmaj7, G7)"
154,"(G7, C)"
155,"(soc, Dm7)"


In [ ]:
fast_tf_idf_df_n2 = pd.concat([fast_tf_idf_df_n2 , pd.DataFrame(np.repeat(0 , len(fast_tf_idf_df_n2)*len(fast)).reshape(len(fast_tf_idf_df_n2) , len(fast)))] , axis = 1)

fast_tf_idf_df_n2.insert(len(fast) + 1 , 'IDF-Count' , np.repeat(0 , len(fast_tf_idf_df_n2)))

fast_tf_idf_df_n2                                                      

,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,2433,2434,2435,2436,2437,2438,2439,2440,2441,IDF-Count
0,"(soc, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(Am, C)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(C, G)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(G, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(Dm, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,"(Fmaj7, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
153,"(Fmaj7, G7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
154,"(G7, C)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
155,"(soc, Dm7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for chord_prog_token in tqdm(prog_token_n2) :

  idf_count = 0

  for i in range(len(fast)) : 

    target_list = fast.loc[i , 'chord_prog_token_n2']

    if chord_prog_token not in target_list :

      tf_score = 0

    else :

      idf_count += 1

      each_chord_prog_token_len = len(target_list)

      count = target_list.count(chord_prog_token)

      tf_score = count/each_chord_prog_token_len

      tf_score = round(tf_score , 4)

    fast_tf_idf_df_n2.loc[fast_tf_idf_df_n2['chord_prog_token_n2'] == chord_prog_token , i] = tf_score

  fast_tf_idf_df_n2.loc[fast_tf_idf_df_n2['chord_prog_token_n2'] == chord_prog_token , 'IDF-Count'] = idf_count

100%|██████████| 157/157 [04:17<00:00,  1.64s/it]


In [ ]:
fast_tf_idf_df_n2

,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,2433,2434,2435,2436,2437,2438,2439,2440,2441,IDF-Count
0,"(soc, Am)",0.1111,0.0,0.1111,0.125,0.0,0.0,0.1111,0.0,0.0,...,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.0000,0.2,1115
1,"(Am, C)",0.2222,0.0,0.2222,0.000,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,186
2,"(C, G)",0.2222,0.0,0.2222,0.000,0.0,0.0,0.2222,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,412
3,"(G, Dm)",0.1111,0.0,0.0000,0.000,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,28
4,"(Dm, Am)",0.1111,0.0,0.0000,0.000,0.2,0.2,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,"(Fmaj7, eoc)",0.0000,0.0,0.0000,0.000,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0000,0.0,19
153,"(Fmaj7, G7)",0.0000,0.0,0.0000,0.000,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0833,0.0,12
154,"(G7, C)",0.0000,0.0,0.0000,0.000,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0833,0.0,12
155,"(soc, Dm7)",0.0000,0.0,0.0000,0.000,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,2


In [ ]:
import math

idf_score_list_n2 = []

for i in tqdm(range(len(fast_tf_idf_df_n2))) : 

  idf_score = math.log(len(fast)/(1 + fast_tf_idf_df_n2.loc[i , 'IDF-Count']))

  idf_score_list_n2.append(idf_score)

fast_tf_idf_df_n2.insert(len(fast) + 2 , 'IDF-Score' , idf_score_list_n2)

fast_tf_idf_df_n2

100%|██████████| 157/157 [00:00<00:00, 33225.98it/s]
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,2434,2435,2436,2437,2438,2439,2440,2441,IDF-Count,IDF-Score
0,"(soc, Am)",0.1111,0.0,0.1111,0.125,0.0,0.0,0.1111,0.0,0.0,...,0.0,0.0,0.2,0.2,0.0,0.0,0.0000,0.2,1115,0.783067
1,"(Am, C)",0.2222,0.0,0.2222,0.000,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,186,2.569464
2,"(C, G)",0.2222,0.0,0.2222,0.000,0.0,0.0,0.2222,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,412,1.777125
3,"(G, Dm)",0.1111,0.0,0.0000,0.000,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,28,4.433277
4,"(Dm, Am)",0.1111,0.0,0.0000,0.000,0.2,0.2,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,200,2.497268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,"(Fmaj7, eoc)",0.0000,0.0,0.0000,0.000,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0000,0.0,19,4.804840
153,"(Fmaj7, G7)",0.0000,0.0,0.0000,0.000,0.0,0.0,0.0000,0.0,0.0,...,0.1,0.0,0.0,0.0,0.0,0.0,0.0833,0.0,12,5.235623
154,"(G7, C)",0.0000,0.0,0.0000,0.000,0.0,0.0,0.0000,0.0,0.0,...,0.1,0.0,0.0,0.0,0.0,0.0,0.0833,0.0,12,5.235623
155,"(soc, Dm7)",0.0000,0.0,0.0000,0.000,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,2,6.701960


In [ ]:
fast_tf_idf_df_n2.to_csv('TF-IDF_fast_n2.csv')

#### ㄷ) TF-IDF Bag of Chord 만들기

In [ ]:
fast_tf_idf_score_n2 = pd.DataFrame({'chord_prog_token_n2' : prog_token_n2})

fast_tf_idf_score_n2 = pd.concat([fast_tf_idf_score_n2 , pd.DataFrame(np.repeat(0 , len(fast_tf_idf_score_n2)*len(fast)).reshape(len(fast_tf_idf_score_n2) , len(fast)))] , axis = 1)
                       
fast_tf_idf_score_n2

,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,2432,2433,2434,2435,2436,2437,2438,2439,2440,2441
0,"(soc, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(Am, C)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(C, G)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(G, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(Dm, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,"(Fmaj7, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
153,"(Fmaj7, G7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
154,"(G7, C)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
155,"(soc, Dm7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in tqdm(range(1 , len(fast) + 1)) :

  fast_tf_idf_score_n2.iloc[: , i] = fast_tf_idf_df_n2.iloc[:,i]*fast_tf_idf_df_n2.loc[:,'IDF-Score']

fast_tf_idf_score_n2

100%|██████████| 2442/2442 [00:02<00:00, 1072.09it/s]


,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,2432,2433,2434,2435,2436,2437,2438,2439,2440,2441
0,"(soc, Am)",0.086999,0.0,0.086999,0.097883,0.000000,0.000000,0.086999,0.0,0.0,...,0.060218,0.0,0.000000,0.0,0.156613,0.156613,0.0,0.0,0.000000,0.156613
1,"(Am, C)",0.570935,0.0,0.570935,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,"(C, G)",0.394877,0.0,0.394877,0.000000,0.000000,0.000000,0.394877,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,"(G, Dm)",0.492537,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,"(Dm, Am)",0.277446,0.0,0.000000,0.000000,0.499454,0.499454,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,"(Fmaj7, eoc)",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.960968,0.000000,0.0,0.0,0.000000,0.000000
153,"(Fmaj7, G7)",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.523562,0.0,0.000000,0.000000,0.0,0.0,0.436127,0.000000
154,"(G7, C)",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.523562,0.0,0.000000,0.000000,0.0,0.0,0.436127,0.000000
155,"(soc, Dm7)",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000


In [ ]:
fast_tf_idf_boc_n2 = pd.DataFrame(fast_tf_idf_score_n2.T)

fast_tf_idf_boc_n2

,0,1,2,3,4,5,6,7,8,9,...,147,148,149,150,151,152,153,154,155,156
chord_prog_token_n2,"(soc, Am)","(Am, C)","(C, G)","(G, Dm)","(Dm, Am)","(G, D)","(D, eoc)","(soc, C)","(C, Gm)","(Gm, C)",...,"(Amaj7, Fmaj7)","(Dm7, eoc)","(C, Csus4)","(Csus4, Cmaj7)","(Am, E7)","(Fmaj7, eoc)","(Fmaj7, G7)","(G7, C)","(soc, Dm7)","(Fmaj7, Gsus4)"
0,0.086999,0.570935,0.394877,0.492537,0.277446,0.381198,0.381198,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10155,0.615524,0.351725,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.086999,0.570935,0.394877,0.0,0.0,0.762396,0.381198,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.097883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2437,0.156613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.436127,0.436127,0.0,0.0


In [ ]:
tf_idf_boc_list_n2 = []

for i in tqdm(range(1, len(fast_tf_idf_boc_n2))) : 

  tf_idf_boc_list_n2.append(list(fast_tf_idf_boc_n2.iloc[i , :]))

100%|██████████| 2442/2442 [00:00<00:00, 9439.64it/s]


In [ ]:
fast_tf_idf_boc_n2 = pd.DataFrame({'id' : fast['id']})

fast_tf_idf_boc_n2.insert(1 , 'TF-IDF Bag of Chords' , tf_idf_boc_list_n2)

fast_tf_idf_boc_n2

,id,TF-IDF Bag of Chords
0,commu00001,"[0.08699868945313564, 0.5709349092027293, 0.39..."
1,commu00004,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10154995..."
2,commu00006,"[0.08699868945313564, 0.5709349092027293, 0.39..."
3,commu00007,"[0.09788331396617421, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,commu00008,"[0.0, 0.0, 0.0, 0.0, 0.4994535493223149, 0.0, ..."
...,...,...
2437,commu11133,"[0.15661330234587875, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2438,commu11135,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2439,commu11136,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2440,commu11140,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
fast_tf_idf_boc_n2.to_csv('TF-IDF_Bag_of_Chords_fast_n2.csv')

### d) Very_fast (bpm > 120)

#### ㄱ) Chord Progression Token 구하기

In [ ]:
very_fast.head()

,audio_key,chord_progressions,chord_list,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]",mid_high,8,150,very_fast,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003
1,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]",mid_low,8,150,very_fast,cinematic,accompaniment,string_ensemble,standard,4/4,113,122,train,commu00010
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, E, F, G#dim, eoc]",mid_high,8,150,very_fast,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00012
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, F, F, F, F, F...","[soc, Am, F, C, G, eoc]",mid,4,130,very_fast,newage,pad,acoustic_piano-2,standard,4/4,55,56,train,commu00013
4,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]",mid,8,150,very_fast,cinematic,main_melody,brass_ensemble,standard,4/4,83,91,train,commu00019


In [ ]:
total_chord_prog_n2 = []

for i in tqdm(range(len(very_fast))) : 

  target_list = very_fast.loc[i , 'chord_list'] 

  chord_prog_list = []

  for j in range(len(target_list)-1) :

    chord_prog = []

    chord_prog.append(target_list[j])
    chord_prog.append(target_list[j+1])

    chord_prog = tuple(chord_prog)

    chord_prog_list.append(chord_prog)

  total_chord_prog_n2.append(chord_prog_list)

total_chord_prog_n2

In [ ]:
very_fast.insert(3 , 'chord_prog_token_n2' , total_chord_prog_n2)

very_fast.head()

,audio_key,chord_progressions,chord_list,chord_prog_token_n2,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]","[(soc, Am), (Am, F), (F, E+), (E+, E), (E, eoc)]",mid_high,8,150,very_fast,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003
1,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]","[(soc, Am), (Am, F), (F, E+), (E+, E), (E, eoc)]",mid_low,8,150,very_fast,cinematic,accompaniment,string_ensemble,standard,4/4,113,122,train,commu00010
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, E, F, G#dim, eoc]","[(soc, Am), (Am, E), (E, F), (F, G#dim), (G#di...",mid_high,8,150,very_fast,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00012
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, F, F, F, F, F...","[soc, Am, F, C, G, eoc]","[(soc, Am), (Am, F), (F, C), (C, G), (G, eoc)]",mid,4,130,very_fast,newage,pad,acoustic_piano-2,standard,4/4,55,56,train,commu00013
4,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]","[(soc, Am), (Am, F), (F, E+), (E+, E), (E, eoc)]",mid,8,150,very_fast,cinematic,main_melody,brass_ensemble,standard,4/4,83,91,train,commu00019


#### ㄴ) TF-IDF 구하기

In [ ]:
prog_token_n2 = []

for i in tqdm(range(len(very_fast))) :

  for chord_token in very_fast.loc[i , 'chord_prog_token_n2'] :

    if chord_token not in prog_token_n2 :

      prog_token_n2.append(chord_token)

100%|██████████| 1889/1889 [00:00<00:00, 26839.11it/s]


In [ ]:
very_fast_tf_idf_df_n2 = pd.DataFrame({'chord_prog_token_n2' : prog_token_n2})

very_fast_tf_idf_df_n2

,chord_prog_token_n2
0,"(soc, Am)"
1,"(Am, F)"
2,"(F, E+)"
3,"(E+, E)"
4,"(E, eoc)"
...,...
175,"(Bdim, Dm)"
176,"(Dm, eoc)"
177,"(Am, Am7)"
178,"(Am7, Am)"


In [ ]:
very_fast_tf_idf_df_n2 = pd.concat([very_fast_tf_idf_df_n2 , pd.DataFrame(np.repeat(0 , len(very_fast_tf_idf_df_n2)*len(very_fast)).reshape(len(very_fast_tf_idf_df_n2) , len(very_fast)))] , axis = 1)

very_fast_tf_idf_df_n2.insert(len(very_fast) + 1 , 'IDF-Count' , np.repeat(0 , len(very_fast_tf_idf_df_n2)))

very_fast_tf_idf_df_n2                                                      

,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,1880,1881,1882,1883,1884,1885,1886,1887,1888,IDF-Count
0,"(soc, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(Am, F)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(F, E+)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(E+, E)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(E, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,"(Bdim, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
176,"(Dm, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
177,"(Am, Am7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
178,"(Am7, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for chord_prog_token in tqdm(prog_token_n2) :

  idf_count = 0

  for i in range(len(very_fast)) : 

    target_list = very_fast.loc[i , 'chord_prog_token_n2']

    if chord_prog_token not in target_list :

      tf_score = 0

    else :

      idf_count += 1

      each_chord_prog_token_len = len(target_list)

      count = target_list.count(chord_prog_token)

      tf_score = count/each_chord_prog_token_len

      tf_score = round(tf_score , 4)

    very_fast_tf_idf_df_n2.loc[very_fast_tf_idf_df_n2['chord_prog_token_n2'] == chord_prog_token , i] = tf_score

  very_fast_tf_idf_df_n2.loc[very_fast_tf_idf_df_n2['chord_prog_token_n2'] == chord_prog_token , 'IDF-Count'] = idf_count

100%|██████████| 180/180 [03:56<00:00,  1.31s/it]


In [ ]:
very_fast_tf_idf_df_n2

,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,1880,1881,1882,1883,1884,1885,1886,1887,1888,IDF-Count
0,"(soc, Am)",0.2,0.2,0.2,0.2,0.2,0.0000,0.2,0.2,0.2,...,0.0556,0.0,0.25,0.2,0.2,0.1429,0.2,0.2,0.2,656
1,"(Am, F)",0.2,0.2,0.0,0.2,0.2,0.2222,0.0,0.0,0.2,...,0.0000,0.0,0.00,0.0,0.0,0.0000,0.2,0.0,0.0,268
2,"(F, E+)",0.2,0.2,0.0,0.0,0.2,0.0000,0.0,0.0,0.2,...,0.0000,0.0,0.00,0.0,0.0,0.0000,0.0,0.0,0.0,71
3,"(E+, E)",0.2,0.2,0.0,0.0,0.2,0.0000,0.0,0.0,0.2,...,0.0000,0.0,0.00,0.0,0.0,0.0000,0.0,0.0,0.0,71
4,"(E, eoc)",0.2,0.2,0.0,0.0,0.2,0.0000,0.0,0.2,0.2,...,0.0000,0.0,0.00,0.0,0.0,0.0000,0.0,0.0,0.0,275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,"(Bdim, Dm)",0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0000,0.0,0.00,0.0,0.0,0.0000,0.0,0.0,0.2,7
176,"(Dm, eoc)",0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0000,0.0,0.00,0.0,0.0,0.0000,0.0,0.0,0.2,7
177,"(Am, Am7)",0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0000,0.0,0.00,0.0,0.0,0.0000,0.0,0.4,0.0,9
178,"(Am7, Am)",0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0000,0.0,0.00,0.0,0.0,0.0000,0.0,0.2,0.0,9


In [ ]:
import math

idf_score_list_n2 = []

for i in tqdm(range(len(very_fast_tf_idf_df_n2))) : 

  idf_score = math.log(len(very_fast)/(1 + very_fast_tf_idf_df_n2.loc[i , 'IDF-Count']))

  idf_score_list_n2.append(idf_score)

very_fast_tf_idf_df_n2.insert(len(very_fast) + 2 , 'IDF-Score' , idf_score_list_n2)

very_fast_tf_idf_df_n2

100%|██████████| 180/180 [00:00<00:00, 36049.02it/s]
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,1881,1882,1883,1884,1885,1886,1887,1888,IDF-Count,IDF-Score
0,"(soc, Am)",0.2,0.2,0.2,0.2,0.2,0.0000,0.2,0.2,0.2,...,0.0,0.25,0.2,0.2,0.1429,0.2,0.2,0.2,656,1.056119
1,"(Am, F)",0.2,0.2,0.0,0.2,0.2,0.2222,0.0,0.0,0.2,...,0.0,0.00,0.0,0.0,0.0000,0.2,0.0,0.0,268,1.949091
2,"(F, E+)",0.2,0.2,0.0,0.0,0.2,0.0000,0.0,0.0,0.2,...,0.0,0.00,0.0,0.0,0.0000,0.0,0.0,0.0,71,3.267137
3,"(E+, E)",0.2,0.2,0.0,0.0,0.2,0.0000,0.0,0.0,0.2,...,0.0,0.00,0.0,0.0,0.0000,0.0,0.0,0.0,71,3.267137
4,"(E, eoc)",0.2,0.2,0.0,0.0,0.2,0.0000,0.0,0.2,0.2,...,0.0,0.00,0.0,0.0,0.0000,0.0,0.0,0.0,275,1.923402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,"(Bdim, Dm)",0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0000,0.0,0.0,0.2,7,5.464361
176,"(Dm, eoc)",0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0000,0.0,0.0,0.2,7,5.464361
177,"(Am, Am7)",0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0000,0.0,0.4,0.0,9,5.241218
178,"(Am7, Am)",0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0000,0.0,0.2,0.0,9,5.241218


In [ ]:
very_fast_tf_idf_df_n2.to_csv('TF-IDF_very_fast_n2.csv')

#### ㄷ) TF-IDF Bag of Chord 만들기

In [ ]:
very_fast_tf_idf_score_n2 = pd.DataFrame({'chord_prog_token_n2' : prog_token_n2})

very_fast_tf_idf_score_n2 = pd.concat([very_fast_tf_idf_score_n2 , pd.DataFrame(np.repeat(0 , len(very_fast_tf_idf_score_n2)*len(very_fast)).reshape(len(very_fast_tf_idf_score_n2) , len(very_fast)))] , axis = 1)
                       
very_fast_tf_idf_score_n2         

,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888
0,"(soc, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(Am, F)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(F, E+)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(E+, E)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(E, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,"(Bdim, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
176,"(Dm, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
177,"(Am, Am7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
178,"(Am7, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in tqdm(range(1 , len(very_fast) + 1)) :

  very_fast_tf_idf_score_n2.iloc[: , i] = very_fast_tf_idf_df_n2.iloc[:,i]*very_fast_tf_idf_df_n2.loc[:,'IDF-Score']

very_fast_tf_idf_score_n2

100%|██████████| 1889/1889 [00:02<00:00, 853.63it/s] 


,chord_prog_token_n2,0,1,2,3,4,5,6,7,8,...,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888
0,"(soc, Am)",0.211224,0.211224,0.211224,0.211224,0.211224,0.000000,0.211224,0.211224,0.211224,...,0.0,0.05872,0.0,0.26403,0.211224,0.211224,0.150919,0.211224,0.211224,0.211224
1,"(Am, F)",0.389818,0.389818,0.000000,0.389818,0.389818,0.433088,0.000000,0.000000,0.389818,...,0.0,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.389818,0.000000,0.000000
2,"(F, E+)",0.653427,0.653427,0.000000,0.000000,0.653427,0.000000,0.000000,0.000000,0.653427,...,0.0,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"(E+, E)",0.653427,0.653427,0.000000,0.000000,0.653427,0.000000,0.000000,0.000000,0.653427,...,0.0,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"(E, eoc)",0.384680,0.384680,0.000000,0.000000,0.384680,0.000000,0.000000,0.384680,0.384680,...,0.0,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,"(Bdim, Dm)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.092872
176,"(Dm, eoc)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.092872
177,"(Am, Am7)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,2.096487,0.000000
178,"(Am7, Am)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,1.048244,0.000000


In [ ]:
very_fast_tf_idf_boc_n2 = pd.DataFrame(very_fast_tf_idf_score_n2.T)

very_fast_tf_idf_boc_n2

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
chord_prog_token_n2,"(soc, Am)","(Am, F)","(F, E+)","(E+, E)","(E, eoc)","(Am, E)","(E, F)","(F, G#dim)","(G#dim, eoc)","(F, C)",...,"(F, Am7)","(Am7, F)","(Dm, Am7)","(F, Am)","(Dm, Bdim)","(Bdim, Dm)","(Dm, eoc)","(Am, Am7)","(Am7, Am)","(Dm, E)"
0,0.211224,0.389818,0.653427,0.653427,0.38468,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.211224,0.389818,0.653427,0.653427,0.38468,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.211224,0.0,0.0,0.0,0.0,0.67083,0.67083,0.579882,0.67083,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.211224,0.389818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.267849,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,0.211224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,0.150919,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.711478,0.0,0.0,0.0,0.0,0.0,0.0
1886,0.211224,0.389818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.942118,0.942118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1887,0.211224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.096487,1.048244,0.0


In [ ]:
tf_idf_boc_list_n2 = []

for i in tqdm(range(1, len(very_fast_tf_idf_boc_n2))) : 

  tf_idf_boc_list_n2.append(list(very_fast_tf_idf_boc_n2.iloc[i , :]))

100%|██████████| 1889/1889 [00:00<00:00, 8494.85it/s]


In [ ]:
very_fast_tf_idf_boc_n2 = pd.DataFrame({'id' : very_fast['id']})

very_fast_tf_idf_boc_n2.insert(1 , 'TF-IDF Bag of Chords' , tf_idf_boc_list_n2)

very_fast_tf_idf_boc_n2

,id,TF-IDF Bag of Chords
0,commu00003,"[0.21122376980337979, 0.38981829757993414, 0.6..."
1,commu00010,"[0.21122376980337979, 0.38981829757993414, 0.6..."
2,commu00012,"[0.21122376980337979, 0.0, 0.0, 0.0, 0.0, 0.67..."
3,commu00013,"[0.21122376980337979, 0.38981829757993414, 0.0..."
4,commu00019,"[0.21122376980337979, 0.38981829757993414, 0.6..."
...,...,...
1884,commu11104,"[0.21122376980337979, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1885,commu11127,"[0.15091938352451484, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1886,commu11142,"[0.21122376980337979, 0.38981829757993414, 0.0..."
1887,commu11143,"[0.21122376980337979, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [ ]:
very_fast_tf_idf_boc_n2.to_csv('TF-IDF_Bag_of_Chords_very_fast_n2.csv')

## 3) BPM 별 각 commu data에 대한 코사인 유사도 계산

In [ ]:
import os

os.chdir('/content/MyDrive/MyDrive/github/data/GW matrix/Bag of Chord')

In [ ]:
def cos_sim(A, B) :

  return dot(A, B)/(norm(A)*norm(B))

### a) Very_slow (bpm <= 60)

In [ ]:
very_slow_tf_idf_boc_n2 = pd.read_csv('TF-IDF_Bag_of_Chords_very_slow_n2.csv')

very_slow_tf_idf_boc_n2.drop('Unnamed: 0' , axis = 1 , inplace = True)

very_slow_tf_idf_boc_n2

,id,TF-IDF Bag of Chords
0,commu00005,"[0.23243654444615658, 0.6877377472371866, 0.52..."
1,commu00011,"[0.23243654444615658, 0.6877377472371866, 0.52..."
2,commu00014,"[0.325313568154173, 0.0, 0.0, 0.0, 0.0, 0.6922..."
3,commu00015,"[0.325313568154173, 0.0, 0.0, 0.0, 0.0, 0.6922..."
4,commu00020,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
1938,commu10843,"[0.8132839203854324, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1939,commu10883,"[0.0, 0.0, 0.0, 0.2326951790003213, 0.15329401..."
1940,commu10899,"[0.0, 0.0, 0.0, 0.2326951790003213, 0.15329401..."
1941,commu11110,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
test = very_slow_tf_idf_boc_n2['TF-IDF Bag of Chords']

for i in range(len(test)) :

  test[i] = test[i].replace("'" , "").replace(" " , "").replace("[" , "").replace("]" , "")
  test[i] = test[i].split(",")

  test[i] = list(map(float , test[i]))

very_slow_tf_idf_boc_n2['TF-IDF Bag of Chords'] = test

In [ ]:
id_ord = list(very_slow_tf_idf_boc_n2['id'])

id_order = CategoricalDtype(categories = id_ord , ordered = True)

very_slow_tf_idf_boc_n2['id'] = very_slow_tf_idf_boc_n2['id'].astype(id_order)

very_slow_tf_idf_boc_n2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1943 entries, 0 to 1942
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    1943 non-null   category
 1   TF-IDF Bag of Chords  1943 non-null   object  
dtypes: category(1), object(1)
memory usage: 98.8+ KB


In [ ]:
id = very_slow_tf_idf_boc_n2['id']

id

0       commu00005
1       commu00011
2       commu00014
3       commu00015
4       commu00020
           ...    
1938    commu10843
1939    commu10883
1940    commu10899
1941    commu11110
1942    commu11134
Name: id, Length: 1943, dtype: category
Categories (1943, object): ['commu00005' < 'commu00011' < 'commu00014' < 'commu00015' < ... <
                            'commu10883' < 'commu10899' < 'commu11110' < 'commu11134']

In [ ]:
very_slow_similarity_df_n2 = pd.DataFrame(np.repeat(0 , len(id)**2).reshape(len(id),len(id)))

very_slow_similarity_df_n2

,0,1,2,3,4,5,6,7,8,9,...,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1939,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1941,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
very_slow_similarity_df_n2.columns = id

very_slow_similarity_df_n2.insert(0 , 'id' , id)

very_slow_similarity_df_n2

id,id,commu00005,commu00011,commu00014,commu00015,commu00020,commu00025,commu00026,commu00044,commu00048,...,commu10826,commu10829,commu10833,commu10835,commu10841,commu10843,commu10883,commu10899,commu11110,commu11134
0,commu00005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,commu00011,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,commu00014,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,commu00015,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,commu00020,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938,commu10843,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1939,commu10883,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1940,commu10899,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1941,commu11110,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for id_1 in tqdm(id) :

  for id_2 in id :

    if id_1 < id_2 :

      A = very_slow_tf_idf_boc_n2.loc[very_slow_tf_idf_boc_n2['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = very_slow_tf_idf_boc_n2.loc[very_slow_tf_idf_boc_n2['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      very_slow_similarity_df_n2.loc[very_slow_similarity_df_n2['id'] == id_1 , id_2] = cos_sim(A , B)

very_slow_similarity_df_n2

100%|██████████| 1943/1943 [41:06<00:00,  1.27s/it]


id,id,commu00005,commu00011,commu00014,commu00015,commu00020,commu00025,commu00026,commu00044,commu00048,...,commu10826,commu10829,commu10833,commu10835,commu10841,commu10843,commu10883,commu10899,commu11110,commu11134
0,commu00005,0,1,0.066611,0.066611,0,0.0,0.066611,0.469432,0.157335,...,0.027499,0.0,0.0,0.0,0.0,0.162768,0.157335,0.157335,0.0,0.029129
1,commu00011,0,0,0.066611,0.066611,0,0.0,0.066611,0.469432,0.157335,...,0.027499,0.0,0.0,0.0,0.0,0.162768,0.157335,0.157335,0.0,0.029129
2,commu00014,0,0,0.000000,1.000000,0,0.0,1.000000,0.071824,0.000000,...,0.203527,0.0,0.0,0.0,0.0,0.409238,0.000000,0.000000,0.0,0.073238
3,commu00015,0,0,0.000000,0.000000,0,0.0,1.000000,0.071824,0.000000,...,0.203527,0.0,0.0,0.0,0.0,0.409238,0.000000,0.000000,0.0,0.073238
4,commu00020,0,0,0.000000,0.000000,0,1.0,0.000000,0.000000,0.007991,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.007991,0.007991,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938,commu10843,0,0,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.178962
1939,commu10883,0,0,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.040805
1940,commu10899,0,0,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.040805
1941,commu11110,0,0,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.105911


In [ ]:
very_slow_similarity_df_n2.to_csv('cos_sim_very_slow_n2.csv')

### b) Slow (60 < bpm <= 90)

In [ ]:
slow_tf_idf_boc_n2 = pd.read_csv('TF-IDF_Bag_of_Chords_slow_n2.csv')

slow_tf_idf_boc_n2.drop('Unnamed: 0' , axis = 1 , inplace = True)

slow_tf_idf_boc_n2

,id,TF-IDF Bag of Chords
0,commu00002,"[0.17077903207839756, 0.4123727278214233, 0.74..."
1,commu00023,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,commu00024,"[0.17077903207839756, 0.4123727278214233, 0.74..."
3,commu00036,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,commu00041,"[0.11820798890034898, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...
4865,commu11125,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4866,commu11128,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4867,commu11137,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4868,commu11138,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
test = slow_tf_idf_boc_n2['TF-IDF Bag of Chords']

for i in range(len(test)) :

  test[i] = test[i].replace("'" , "").replace(" " , "").replace("[", "").replace("]", "")
  test[i] = test[i].split(",")

  test[i] = list(map(float , test[i]))

slow_tf_idf_boc_n2['TF-IDF Bag of Chords'] = test

In [ ]:
id_ord = list(slow_tf_idf_boc_n2['id'])

id_order = CategoricalDtype(categories = id_ord , ordered = True)

slow_tf_idf_boc_n2['id'] = slow_tf_idf_boc_n2['id'].astype(id_order)

slow_tf_idf_boc_n2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4870 entries, 0 to 4869
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    4870 non-null   category
 1   TF-IDF Bag of Chords  4870 non-null   object  
dtypes: category(1), object(1)
memory usage: 214.8+ KB


In [ ]:
id = slow_tf_idf_boc_n2['id']

id

0       commu00002
1       commu00023
2       commu00024
3       commu00036
4       commu00041
           ...    
4865    commu11125
4866    commu11128
4867    commu11137
4868    commu11138
4869    commu11139
Name: id, Length: 4870, dtype: category
Categories (4870, object): ['commu00002' < 'commu00023' < 'commu00024' < 'commu00036' < ... <
                            'commu11128' < 'commu11137' < 'commu11138' < 'commu11139']

In [ ]:
slow_similarity_df_n2 = pd.DataFrame(np.repeat(0 , len(id)**2).reshape(len(id),len(id)))

slow_similarity_df_n2

,0,1,2,3,4,5,6,7,8,9,...,4860,4861,4862,4863,4864,4865,4866,4867,4868,4869
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4866,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4867,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4868,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
id_list_1 = slow_tf_idf_boc_n2['id'][:2435]
id_list_2 = slow_tf_idf_boc_n2['id'][2435:]

In [ ]:
slow_1 = slow_similarity_df_n2.iloc[:2435 , :2435].copy()
slow_2 = slow_similarity_df_n2.iloc[:2435 , :2435].copy()
slow_3 = slow_similarity_df_n2.iloc[:2435 , :2435].copy()
slow_4 = slow_similarity_df_n2.iloc[:2435 , :2435].copy()

id_list_1 = list(slow_tf_idf_boc_n2['id'][:2435])
id_list_2 = list(slow_tf_idf_boc_n2['id'][2435:])

slow_1.columns = id_list_1
slow_1.insert(0 , 'id' , id_list_1)

slow_2.columns = id_list_2
slow_2.insert(0 , 'id' , id_list_1)

slow_3.columns = id_list_1
slow_3.insert(0 , 'id' , id_list_2)

slow_4.columns = id_list_2
slow_4.insert(0 , 'id' , id_list_2)

In [ ]:
for id_1 in tqdm(id_list_1) :

  for id_2 in id_list_1 :

    if id_1 < id_2 :

      A = slow_tf_idf_boc_n2.loc[slow_tf_idf_boc_n2['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = slow_tf_idf_boc_n2.loc[slow_tf_idf_boc_n2['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      slow_1.loc[slow_1['id'] == id_1 , id_2] = cos_sim(A , B)

slow_1

100%|██████████| 2435/2435 [1:25:13<00:00,  2.10s/it]


,id,commu00002,commu00023,commu00024,commu00036,commu00041,commu00042,commu00043,commu00046,commu00054,...,commu05498,commu05501,commu05502,commu05503,commu05504,commu05505,commu05506,commu05507,commu05508,commu05514
0,commu00002,0,0,1.0,0,0.187877,0.028159,0.105506,0.413501,0.098816,...,0.000000,0.066545,0.000000,0.0,0.0,0.082538,0.000000,0.0,0.000000,0.000000
1,commu00023,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.074527,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
2,commu00024,0,0,0.0,0,0.187877,0.028159,0.105506,0.413501,0.098816,...,0.000000,0.066545,0.000000,0.0,0.0,0.082538,0.000000,0.0,0.000000,0.000000
3,commu00036,0,0,0.0,0,0.265128,0.000000,0.000000,0.000000,0.000000,...,0.330473,0.000000,0.330473,0.0,0.0,0.414961,1.000000,0.0,0.227122,0.000000
4,commu00041,0,0,0.0,0,0.000000,0.000000,0.392648,0.122198,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.391397,0.265128,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430,commu05505,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.414961,0.0,0.190330,0.000000
2431,commu05506,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.227122,0.000000
2432,commu05507,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.210511
2433,commu05508,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000


In [ ]:
slow_1.to_csv('slow_1.csv')

In [ ]:
for id_1 in tqdm(id_list_1) :

  for id_2 in id_list_2 :

    if id_1 < id_2 :

      A = slow_tf_idf_boc_n2.loc[slow_tf_idf_boc_n2['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = slow_tf_idf_boc_n2.loc[slow_tf_idf_boc_n2['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      slow_2.loc[slow_2['id'] == id_1 , id_2] = cos_sim(A , B)

slow_2

100%|██████████| 2435/2435 [2:36:19<00:00,  3.85s/it]


,id,commu05516,commu05517,commu05519,commu05520,commu05521,commu05523,commu05525,commu05526,commu05528,...,commu11114,commu11116,commu11117,commu11120,commu11124,commu11125,commu11128,commu11137,commu11138,commu11139
0,commu00002,0.000000,0.000000,0.082538,0.000000,0.000000,0.000000,0.082538,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,commu00023,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,commu00024,0.000000,0.000000,0.082538,0.000000,0.000000,0.000000,0.082538,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,commu00036,1.000000,0.078468,0.414961,0.330473,0.227122,0.235209,0.414961,0.200957,0.000000,...,0.058736,0.031545,0.010629,0.031545,0.058736,0.058736,0.000000,0.058736,0.058736,0.000000
4,commu00041,0.265128,0.000000,0.391397,0.000000,0.000000,0.000000,0.391397,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430,commu05505,0.414961,0.099961,1.000000,0.021504,0.190330,0.214550,1.000000,0.118105,0.000000,...,0.074825,0.040186,0.013540,0.040186,0.074825,0.074825,0.000000,0.074825,0.074825,0.000000
2431,commu05506,1.000000,0.078468,0.414961,0.330473,0.227122,0.235209,0.414961,0.200957,0.000000,...,0.058736,0.031545,0.010629,0.031545,0.058736,0.058736,0.000000,0.058736,0.058736,0.000000
2432,commu05507,0.000000,0.054110,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.210511,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.723577,0.000000,0.000000,0.723577
2433,commu05508,0.227122,0.062043,0.190330,0.157151,1.000000,0.011672,0.190330,0.324078,0.000000,...,0.046441,0.024942,0.008404,0.024942,0.046441,0.046441,0.000000,0.046441,0.046441,0.000000


In [ ]:
slow_2.to_csv('slow_2.csv')

In [ ]:
slow_3.to_csv('slow_3.csv')

In [ ]:
for id_1 in tqdm(id_list_2) :

  for id_2 in id_list_2 :

    if id_1 < id_2 :

      A = slow_tf_idf_boc_n2.loc[slow_tf_idf_boc_n2['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = slow_tf_idf_boc_n2.loc[slow_tf_idf_boc_n2['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      slow_4.loc[slow_4['id'] == id_1 , id_2] = cos_sim(A , B)

slow_4

100%|██████████| 2435/2435 [1:15:37<00:00,  1.86s/it]


,id,commu05516,commu05517,commu05519,commu05520,commu05521,commu05523,commu05525,commu05526,commu05528,...,commu11114,commu11116,commu11117,commu11120,commu11124,commu11125,commu11128,commu11137,commu11138,commu11139
0,commu05516,0,0.078468,0.414961,0.330473,0.227122,0.235209,0.414961,0.200957,0.00000,...,0.058736,0.031545,0.010629,0.031545,0.058736,0.058736,0.0,0.058736,0.058736,0.0
1,commu05517,0,0.000000,0.099961,0.012780,0.062043,0.011176,0.099961,0.041300,0.10368,...,0.044471,0.023884,0.008047,0.023884,0.044471,0.044471,0.0,0.044471,0.044471,0.0
2,commu05519,0,0.000000,0.000000,0.021504,0.190330,0.214550,1.000000,0.118105,0.00000,...,0.074825,0.040186,0.013540,0.040186,0.074825,0.074825,0.0,0.074825,0.074825,0.0
3,commu05520,0,0.000000,0.000000,0.000000,0.157151,0.191362,0.021504,0.493246,0.00000,...,0.054126,0.029069,0.009795,0.029069,0.054126,0.054126,0.0,0.054126,0.054126,0.0
4,commu05521,0,0.000000,0.000000,0.000000,0.000000,0.011672,0.190330,0.324078,0.00000,...,0.046441,0.024942,0.008404,0.024942,0.046441,0.046441,0.0,0.046441,0.046441,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430,commu11125,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,1.000000,0.0
2431,commu11128,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.0
2432,commu11137,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.0
2433,commu11138,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [ ]:
slow_4.to_csv('slow_4.csv')

In [ ]:
slow_12 = pd.concat([slow_1 , slow_2] , axis = 1)
slow_34 = pd.concat([slow_3 , slow_4] , axis = 1)

slow_similarity_df_n2 = pd.concat([slow_12 , slow_34])

slow_similarity_df_n2

,id,commu00002,commu00023,commu00024,commu00036,commu00041,commu00042,commu00043,commu00046,commu00054,...,commu11114,commu11116,commu11117,commu11120,commu11124,commu11125,commu11128,commu11137,commu11138,commu11139
0,commu00002,0,0,1.0,0,0.187877,0.028159,0.105506,0.413501,0.098816,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,commu00023,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,commu00024,0,0,0.0,0,0.187877,0.028159,0.105506,0.413501,0.098816,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
3,commu00036,0,0,0.0,0,0.265128,0.000000,0.000000,0.000000,0.000000,...,0.058736,0.031545,0.010629,0.031545,0.058736,0.058736,0.0,0.058736,0.058736,0.0
4,commu00041,0,0,0.0,0,0.000000,0.000000,0.392648,0.122198,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430,commu11125,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,1.000000,0.0
2431,commu11128,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.0
2432,commu11137,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.0
2433,commu11138,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [ ]:
slow_similarity_df_n2.to_csv('cos_sim_slow_n2.csv')

### C) Fast (90 < bpm <= 120)

In [ ]:
fast_tf_idf_boc_n2 = pd.read_csv('TF-IDF_Bag_of_Chords_fast_n2.csv')

fast_tf_idf_boc_n2.drop('Unnamed: 0' , axis = 1 , inplace = True)

fast_tf_idf_boc_n2

In [ ]:
test = fast_tf_idf_boc_n2['TF-IDF Bag of Chords']

for i in range(len(test)) :

  test[i] = test[i].replace("'" , "").replace(" " , "").replace("[", "").replace("]", "")
  test[i] = test[i].split(",")

  test[i] = list(map(float , test[i]))

fast_tf_idf_boc_n2['TF-IDF Bag of Chords'] = test

In [ ]:
id_ord = list(fast_tf_idf_boc_n2['id'])

id_order = CategoricalDtype(categories = id_ord , ordered = True)

fast_tf_idf_boc_n2['id'] = fast_tf_idf_boc_n2['id'].astype(id_order)

fast_tf_idf_boc_n2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2442 entries, 0 to 2441
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    2442 non-null   category
 1   TF-IDF Bag of Chords  2442 non-null   object  
dtypes: category(1), object(1)
memory usage: 107.6+ KB


In [ ]:
id = fast_tf_idf_boc_n2['id']

id

0       commu00001
1       commu00004
2       commu00006
3       commu00007
4       commu00008
           ...    
2437    commu11133
2438    commu11135
2439    commu11136
2440    commu11140
2441    commu11141
Name: id, Length: 2442, dtype: category
Categories (2442, object): ['commu00001' < 'commu00004' < 'commu00006' < 'commu00007' < ... <
                            'commu11135' < 'commu11136' < 'commu11140' < 'commu11141']

In [ ]:
fast_similarity_df_n2 = pd.DataFrame(np.repeat(0 , len(id)**2).reshape(len(id),len(id)))

fast_similarity_df_n2

,0,1,2,3,4,5,6,7,8,9,...,2432,2433,2434,2435,2436,2437,2438,2439,2440,2441
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2439,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2440,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
fast_similarity_df_n2.columns = id

fast_similarity_df_n2.insert(0 , 'id' , id)

fast_similarity_df_n2

id,id,commu00001,commu00004,commu00006,commu00007,commu00008,commu00009,commu00016,commu00017,commu00018,...,commu11126,commu11129,commu11130,commu11131,commu11132,commu11133,commu11135,commu11136,commu11140,commu11141
0,commu00001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,commu00004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,commu00006,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,commu00007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,commu00008,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2437,commu11133,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2438,commu11135,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2439,commu11136,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2440,commu11140,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for id_1 in tqdm(id) :

  for id_2 in id :

    if id_1 < id_2 :

      A = fast_tf_idf_boc_n2.loc[fast_tf_idf_boc_n2['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = fast_tf_idf_boc_n2.loc[fast_tf_idf_boc_n2['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      fast_similarity_df_n2.loc[fast_similarity_df_n2['id'] == id_1 , id_2] = cos_sim(A , B)

fast_similarity_df_n2

100%|██████████| 2442/2442 [1:04:21<00:00,  1.58s/it]


id,id,commu00001,commu00004,commu00006,commu00007,commu00008,commu00009,commu00016,commu00017,commu00018,...,commu11126,commu11129,commu11130,commu11131,commu11132,commu11133,commu11135,commu11136,commu11140,commu11141
0,commu00001,0,0,0.769992,0.005205,0.098310,0.116638,0.221978,0.000000,0.000000,...,0.004316,0.000000,0.000000,0.000000,0.007692,0.013735,0.000000,0.000000,0.000000,0.009842
1,commu00004,0,0,0.000000,0.000000,0.013891,0.020972,0.022816,1.000000,0.055334,...,0.000000,0.007924,0.012304,0.000000,0.000000,0.000000,0.000000,0.000000,0.011250,0.021852
2,commu00006,0,0,0.000000,0.127433,0.000000,0.000000,0.203107,0.000000,0.000000,...,0.003949,0.000000,0.000000,0.000000,0.007038,0.012567,0.000000,0.000000,0.000000,0.009005
3,commu00007,0,0,0.000000,0.000000,0.000000,0.000000,0.007771,0.000000,0.000000,...,0.003264,0.000000,0.000000,0.018864,0.005817,0.010387,0.018864,0.018864,0.000000,0.007443
4,commu00008,0,0,0.000000,0.000000,0.000000,0.714387,0.000000,0.013891,0.028951,...,0.000000,0.010551,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2437,commu11133,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019640
2438,commu11135,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.419259,0.000000
2439,commu11136,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.419259,0.000000
2440,commu11140,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015251


In [ ]:
fast_similarity_df_n2.to_csv('cos_sim_fast_n2.csv')

### D) Very_fast (120 < bpm)

In [ ]:
very_fast_tf_idf_boc_n2 = pd.read_csv('TF-IDF_Bag_of_Chords_very_fast_n2.csv')

very_fast_tf_idf_boc_n2.drop('Unnamed: 0' , axis = 1 , inplace = True)

very_fast_tf_idf_boc_n2

,id,TF-IDF Bag of Chords
0,commu00003,"[0.21122376980337979, 0.38981829757993414, 0.6..."
1,commu00010,"[0.21122376980337979, 0.38981829757993414, 0.6..."
2,commu00012,"[0.21122376980337979, 0.0, 0.0, 0.0, 0.0, 0.67..."
3,commu00013,"[0.21122376980337979, 0.38981829757993414, 0.0..."
4,commu00019,"[0.21122376980337979, 0.38981829757993414, 0.6..."
...,...,...
1884,commu11104,"[0.21122376980337979, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1885,commu11127,"[0.15091938352451484, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1886,commu11142,"[0.21122376980337979, 0.38981829757993414, 0.0..."
1887,commu11143,"[0.21122376980337979, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [ ]:
test = very_fast_tf_idf_boc_n2['TF-IDF Bag of Chords']

for i in range(len(test)) :

  test[i] = test[i].replace("'" , "").replace(" " , "").replace("[", "").replace("]", "")
  test[i] = test[i].split(",")

  test[i] = list(map(float , test[i]))

very_fast_tf_idf_boc_n2['TF-IDF Bag of Chords'] = test

In [ ]:
id_ord = list(very_fast_tf_idf_boc_n2['id'])

id_order = CategoricalDtype(categories = id_ord , ordered = True)

very_fast_tf_idf_boc_n2['id'] = very_fast_tf_idf_boc_n2['id'].astype(id_order)

very_fast_tf_idf_boc_n2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    1889 non-null   category
 1   TF-IDF Bag of Chords  1889 non-null   object  
dtypes: category(1), object(1)
memory usage: 97.9+ KB


In [ ]:
very_fast_tf_idf_boc_n2['id']

0       commu00003
1       commu00010
2       commu00012
3       commu00013
4       commu00019
           ...    
1884    commu11104
1885    commu11127
1886    commu11142
1887    commu11143
1888    commu11144
Name: id, Length: 1889, dtype: category
Categories (1889, object): ['commu00003' < 'commu00010' < 'commu00012' < 'commu00013' < ... <
                            'commu11127' < 'commu11142' < 'commu11143' < 'commu11144']

In [ ]:
id = very_fast_tf_idf_boc_n2['id']

id

0       commu00003
1       commu00010
2       commu00012
3       commu00013
4       commu00019
           ...    
1884    commu11104
1885    commu11127
1886    commu11142
1887    commu11143
1888    commu11144
Name: id, Length: 1889, dtype: category
Categories (1889, object): ['commu00003' < 'commu00010' < 'commu00012' < 'commu00013' < ... <
                            'commu11127' < 'commu11142' < 'commu11143' < 'commu11144']

In [ ]:
very_fast_similarity_df_n2 = pd.DataFrame(np.repeat(0 , len(id)**2).reshape(len(id),len(id)))

very_fast_similarity_df_n2

,0,1,2,3,4,5,6,7,8,9,...,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1885,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
very_fast_similarity_df_n2.columns = id

very_fast_similarity_df_n2.insert(0 , 'id' , id)

very_fast_similarity_df_n2

id,id,commu00003,commu00010,commu00012,commu00013,commu00019,commu00021,commu00027,commu00047,commu00052,...,commu11091,commu11093,commu11094,commu11098,commu11099,commu11104,commu11127,commu11142,commu11143,commu11144
0,commu00003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,commu00010,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,commu00012,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,commu00013,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,commu00019,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,commu11104,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1885,commu11127,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1886,commu11142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,commu11143,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for id_1 in tqdm(id) :

  for id_2 in id :

    if id_1 < id_2 :

      A = very_fast_tf_idf_boc_n2.loc[very_fast_tf_idf_boc_n2['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = very_fast_tf_idf_boc_n2.loc[very_fast_tf_idf_boc_n2['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      very_fast_similarity_df_n2.loc[very_fast_similarity_df_n2['id'] == id_1 , id_2] = cos_sim(A , B)

very_fast_similarity_df_n2

100%|██████████| 1889/1889 [42:03<00:00,  1.34s/it]


id,id,commu00003,commu00010,commu00012,commu00013,commu00019,commu00021,commu00027,commu00047,commu00052,...,commu11091,commu11093,commu11094,commu11098,commu11099,commu11104,commu11127,commu11142,commu11143,commu11144
0,commu00003,0,1,0.030976,0.259880,1.000000,0.205477,0.030976,0.153012,1.000000,...,0.000000,0.012335,0.000000,0.027745,0.032986,0.032986,0.021124,0.121286,0.016348,0.020463
1,commu00010,0,0,0.030976,0.259880,1.000000,0.205477,0.030976,0.153012,1.000000,...,0.000000,0.012335,0.000000,0.027745,0.032986,0.032986,0.021124,0.121286,0.016348,0.020463
2,commu00012,0,0,0.000000,0.049081,0.030976,0.000000,1.000000,0.029495,0.030976,...,0.000000,0.010264,0.000000,0.023087,0.027448,0.027448,0.017578,0.022906,0.013604,0.017028
3,commu00013,0,0,0.000000,0.000000,0.259880,0.546454,0.049081,0.056163,0.259880,...,0.050295,0.019544,0.076463,0.043961,0.052265,0.052265,0.033470,0.192173,0.025903,0.032423
4,commu00019,0,0,0.000000,0.000000,0.000000,0.205477,0.030976,0.153012,1.000000,...,0.000000,0.012335,0.000000,0.027745,0.032986,0.032986,0.021124,0.121286,0.016348,0.020463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,commu11104,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.536152,0.024392,0.014486,0.018132
1885,commu11127,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.092653,0.009277,0.011612
1886,commu11142,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012089,0.015132
1887,commu11143,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008987


In [ ]:
very_fast_similarity_df_n2.to_csv('cos_sim_very_fast_n2.csv')

# 3. n = 3

In [ ]:
data.head()

,audio_key,chord_progressions,chord_list,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, C, C, C, C, C...","[soc, Am, C, G, Dm, Am, C, G, D, eoc]",mid,8,120,fast,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001
1,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, G7, C, Am, F, G, C, eoc]",mid_low,8,80,slow,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]",mid_high,8,150,very_fast,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003
3,cmajor,"[C, C, C, C, C, C, C, C, Gm, Gm, Gm, Gm, Gm, G...","[soc, C, Gm, C, Gm, Dm, A#, F, G, eoc]",mid,8,110,fast,cinematic,pad,choir,standard,4/4,45,46,train,commu00004
4,aminor,"[Am, Am, Am, Am, Em, Em, Em, Em, F, F, F, F, F...","[soc, Am, Em, F, Am, Em, F, eoc]",mid_low,4,60,very_slow,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005


## 1) BPM 구분
### a) very_slow ( ~ 60bpm)
### b) slow (61 ~ 90 bpm)
### c) fast (91 ~ 120 bpm)
### d) very fast(121 bpm ~ )

In [ ]:
data.head()

,audio_key,chord_progressions,chord_list,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, C, C, C, C, C...","[soc, Am, C, G, Dm, Am, C, G, D, eoc]",mid,8,120,fast,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001
1,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, G7, C, Am, F, G, C, eoc]",mid_low,8,80,slow,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]",mid_high,8,150,very_fast,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003
3,cmajor,"[C, C, C, C, C, C, C, C, Gm, Gm, Gm, Gm, Gm, G...","[soc, C, Gm, C, Gm, Dm, A#, F, G, eoc]",mid,8,110,fast,cinematic,pad,choir,standard,4/4,45,46,train,commu00004
4,aminor,"[Am, Am, Am, Am, Em, Em, Em, Em, F, F, F, F, F...","[soc, Am, Em, F, Am, Em, F, eoc]",mid_low,4,60,very_slow,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005


In [ ]:
very_slow = data[data['bpm_group'] == 'very_slow']
very_slow.reset_index(drop = True , inplace = True)

slow = data[data['bpm_group'] == 'slow']
slow.reset_index(drop = True , inplace = True)

fast = data[data['bpm_group'] == 'fast']
fast.reset_index(drop = True , inplace = True)

very_fast = data[data['bpm_group'] == 'very_fast']
very_fast.reset_index(drop = True , inplace = True)

print('매우 느리게 : ' , len(very_slow))
print('-'*30)
print('느리게 : ' , len(slow))
print('-'*30)
print('빠르게 : ' , len(fast))
print('-'*30)
print('매우 빠르게 : ' , len(very_fast))

매우 느리게 :  1943
------------------------------
느리게 :  4870
------------------------------
빠르게 :  2442
------------------------------
매우 빠르게 :  1889


## 2) BPM별 TF-IDF 구하기

### a) Very_slow (bpm <= 60)

#### ㄱ) Chord Progression Token 구하기

In [ ]:
very_slow.head()

,audio_key,chord_progressions,chord_list,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Em, Em, Em, Em, F, F, F, F, F...","[soc, Am, Em, F, Am, Em, F, eoc]",mid_low,4,60,very_slow,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005
1,aminor,"[Am, Am, Am, Am, Em, Em, Em, Em, F, F, F, F, F...","[soc, Am, Em, F, Am, Em, F, eoc]",mid_high,4,60,very_slow,cinematic,main_melody,string_ensemble,standard,4/4,21,22,train,commu00011
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Dm, Dm, Dm, D...","[soc, Am, Dm, E, Am, eoc]",mid_high,4,60,very_slow,cinematic,main_melody,string_ensemble,standard,4/4,41,42,train,commu00014
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Dm, Dm, Dm, D...","[soc, Am, Dm, E, Am, eoc]",mid_low,4,60,very_slow,cinematic,pad,string_ensemble,standard,4/4,22,24,train,commu00015
4,cmajor,"[C, C, C, C, C, C, C, C, Dm7, Dm7, Dm7, Dm7, D...","[soc, C, Dm7, C, A#, C, Dm7, C, A#, eoc]",mid,8,50,very_slow,newage,pad,brass_ensemble,standard,4/4,81,82,train,commu00020


In [ ]:
total_chord_prog_n3 = []

for i in tqdm(range(len(very_slow))) : 

  target_list = very_slow.loc[i , 'chord_list'] 

  chord_prog_list = []

  for j in range(len(target_list)-2) :

    chord_prog = []

    chord_prog.append(target_list[j])
    chord_prog.append(target_list[j+1])
    chord_prog.append(target_list[j+2])

    chord_prog = tuple(chord_prog)

    chord_prog_list.append(chord_prog)

  total_chord_prog_n3.append(chord_prog_list)

total_chord_prog_n3

100%|██████████| 1943/1943 [00:00<00:00, 26998.44it/s]


[[('soc', 'Am', 'Em'),
  ('Am', 'Em', 'F'),
  ('Em', 'F', 'Am'),
  ('F', 'Am', 'Em'),
  ('Am', 'Em', 'F'),
  ('Em', 'F', 'eoc')],
 [('soc', 'Am', 'Em'),
  ('Am', 'Em', 'F'),
  ('Em', 'F', 'Am'),
  ('F', 'Am', 'Em'),
  ('Am', 'Em', 'F'),
  ('Em', 'F', 'eoc')],
 [('soc', 'Am', 'Dm'),
  ('Am', 'Dm', 'E'),
  ('Dm', 'E', 'Am'),
  ('E', 'Am', 'eoc')],
 [('soc', 'Am', 'Dm'),
  ('Am', 'Dm', 'E'),
  ('Dm', 'E', 'Am'),
  ('E', 'Am', 'eoc')],
 [('soc', 'C', 'Dm7'),
  ('C', 'Dm7', 'C'),
  ('Dm7', 'C', 'A#'),
  ('C', 'A#', 'C'),
  ('A#', 'C', 'Dm7'),
  ('C', 'Dm7', 'C'),
  ('Dm7', 'C', 'A#'),
  ('C', 'A#', 'eoc')],
 [('soc', 'C', 'Dm7'),
  ('C', 'Dm7', 'C'),
  ('Dm7', 'C', 'A#'),
  ('C', 'A#', 'C'),
  ('A#', 'C', 'Dm7'),
  ('C', 'Dm7', 'C'),
  ('Dm7', 'C', 'A#'),
  ('C', 'A#', 'eoc')],
 [('soc', 'Am', 'Dm'),
  ('Am', 'Dm', 'E'),
  ('Dm', 'E', 'Am'),
  ('E', 'Am', 'eoc')],
 [('soc', 'Am', 'Em'),
  ('Am', 'Em', 'C'),
  ('Em', 'C', 'F'),
  ('C', 'F', 'Am'),
  ('F', 'Am', 'G'),
  ('Am', 'G', 'C'),
  ('

In [ ]:
very_slow.insert(3 , 'chord_prog_token_n3' , total_chord_prog_n3)

very_slow.head()

,audio_key,chord_progressions,chord_list,chord_prog_token_n3,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Em, Em, Em, Em, F, F, F, F, F...","[soc, Am, Em, F, Am, Em, F, eoc]","[(soc, Am, Em), (Am, Em, F), (Em, F, Am), (F, ...",mid_low,4,60,very_slow,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005
1,aminor,"[Am, Am, Am, Am, Em, Em, Em, Em, F, F, F, F, F...","[soc, Am, Em, F, Am, Em, F, eoc]","[(soc, Am, Em), (Am, Em, F), (Em, F, Am), (F, ...",mid_high,4,60,very_slow,cinematic,main_melody,string_ensemble,standard,4/4,21,22,train,commu00011
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Dm, Dm, Dm, D...","[soc, Am, Dm, E, Am, eoc]","[(soc, Am, Dm), (Am, Dm, E), (Dm, E, Am), (E, ...",mid_high,4,60,very_slow,cinematic,main_melody,string_ensemble,standard,4/4,41,42,train,commu00014
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Dm, Dm, Dm, D...","[soc, Am, Dm, E, Am, eoc]","[(soc, Am, Dm), (Am, Dm, E), (Dm, E, Am), (E, ...",mid_low,4,60,very_slow,cinematic,pad,string_ensemble,standard,4/4,22,24,train,commu00015
4,cmajor,"[C, C, C, C, C, C, C, C, Dm7, Dm7, Dm7, Dm7, D...","[soc, C, Dm7, C, A#, C, Dm7, C, A#, eoc]","[(soc, C, Dm7), (C, Dm7, C), (Dm7, C, A#), (C,...",mid,8,50,very_slow,newage,pad,brass_ensemble,standard,4/4,81,82,train,commu00020


#### ㄴ) TF-IDF 구하기

In [ ]:
prog_token_n3 = []

for i in tqdm(range(len(very_slow))) :

  for chord_token in very_slow.loc[i , 'chord_prog_token_n3'] :

    if chord_token not in prog_token_n3 :

      prog_token_n3.append(chord_token)

100%|██████████| 1943/1943 [00:00<00:00, 10232.87it/s]


In [ ]:
very_slow_tf_idf_df_n3 = pd.DataFrame({'chord_prog_token_n3' : prog_token_n3})

very_slow_tf_idf_df_n3

,chord_prog_token_n3
0,"(soc, Am, Em)"
1,"(Am, Em, F)"
2,"(Em, F, Am)"
3,"(F, Am, Em)"
4,"(Em, F, eoc)"
...,...
320,"(Dm7, Am7, Em7)"
321,"(Em7, Dm7, eoc)"
322,"(C, G, Dm)"
323,"(G, Dm, G)"


In [ ]:
very_slow_tf_idf_df_n3 = pd.concat([very_slow_tf_idf_df_n3 , pd.DataFrame(np.repeat(0 , len(very_slow_tf_idf_df_n3)*len(very_slow)).reshape(len(very_slow_tf_idf_df_n3) , len(very_slow)))] , axis = 1)

very_slow_tf_idf_df_n3.insert(len(very_slow) + 1 , 'IDF-Count' , np.repeat(0 , len(very_slow_tf_idf_df_n3)))

very_slow_tf_idf_df_n3                                                      

,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,1934,1935,1936,1937,1938,1939,1940,1941,1942,IDF-Count
0,"(soc, Am, Em)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(Am, Em, F)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(Em, F, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(F, Am, Em)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(Em, F, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,"(Dm7, Am7, Em7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
321,"(Em7, Dm7, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
322,"(C, G, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
323,"(G, Dm, G)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for chord_prog_token in tqdm(prog_token_n3) :

  idf_count = 0

  for i in range(len(very_slow)) : 

    target_list = very_slow.loc[i , 'chord_prog_token_n3']

    if chord_prog_token not in target_list :

      tf_score = 0

    else :

      idf_count += 1

      each_chord_prog_token_len = len(target_list)

      count = target_list.count(chord_prog_token)

      tf_score = count/each_chord_prog_token_len

      tf_score = round(tf_score , 4)

    very_slow_tf_idf_df_n3.loc[very_slow_tf_idf_df_n3['chord_prog_token_n3'] == chord_prog_token , i] = tf_score

  very_slow_tf_idf_df_n3.loc[very_slow_tf_idf_df_n3['chord_prog_token_n3'] == chord_prog_token , 'IDF-Count'] = idf_count

100%|██████████| 325/325 [06:45<00:00,  1.25s/it]


In [ ]:
very_slow_tf_idf_df_n3

,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,1934,1935,1936,1937,1938,1939,1940,1941,1942,IDF-Count
0,"(soc, Am, Em)",0.1667,0.1667,0.0,0.0,0.0,0.0,0.0,0.125,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0000,137
1,"(Am, Em, F)",0.3333,0.3333,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0000,134
2,"(Em, F, Am)",0.1667,0.1667,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0000,39
3,"(F, Am, Em)",0.1667,0.1667,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0000,39
4,"(Em, F, eoc)",0.1667,0.1667,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0000,175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,"(Dm7, Am7, Em7)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0000,2
321,"(Em7, Dm7, eoc)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0000,2
322,"(C, G, Dm)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0909,6
323,"(G, Dm, G)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0909,6


In [ ]:
import math

idf_score_list_n3 = []

for i in tqdm(range(len(very_slow_tf_idf_df_n3))) : 

  idf_score = math.log(len(very_slow)/(1 + very_slow_tf_idf_df_n3.loc[i , 'IDF-Count']))

  idf_score_list_n3.append(idf_score)

very_slow_tf_idf_df_n3.insert(len(very_slow) + 2 , 'IDF-Score' , idf_score_list_n3)

very_slow_tf_idf_df_n3

100%|██████████| 325/325 [00:00<00:00, 42047.84it/s]
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,1935,1936,1937,1938,1939,1940,1941,1942,IDF-Count,IDF-Score
0,"(soc, Am, Em)",0.1667,0.1667,0.0,0.0,0.0,0.0,0.0,0.125,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0000,137,2.644735
1,"(Am, Em, F)",0.3333,0.3333,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0000,134,2.666714
2,"(Em, F, Am)",0.1667,0.1667,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0000,39,3.883109
3,"(F, Am, Em)",0.1667,0.1667,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0000,39,3.883109
4,"(Em, F, eoc)",0.1667,0.1667,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0000,175,2.401504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,"(Dm7, Am7, Em7)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0000,2,6.473376
321,"(Em7, Dm7, eoc)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0000,2,6.473376
322,"(C, G, Dm)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0909,6,5.626078
323,"(G, Dm, G)",0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0909,6,5.626078


In [ ]:
very_slow_tf_idf_df_n3.to_csv('TF-IDF_very_slow_n3.csv')

#### ㄷ) TF-IDF Bag of Chord 만들기

In [ ]:
very_slow_tf_idf_score_n3 = pd.DataFrame({'chord_prog_token_n3' : prog_token_n3})

very_slow_tf_idf_score_n3 = pd.concat([very_slow_tf_idf_score_n3 , pd.DataFrame(np.repeat(0 , len(very_slow_tf_idf_score_n3)*len(very_slow)).reshape(len(very_slow_tf_idf_score_n3) , len(very_slow)))] , axis = 1)
                       
very_slow_tf_idf_score_n3

,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942
0,"(soc, Am, Em)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(Am, Em, F)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(Em, F, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(F, Am, Em)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(Em, F, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,"(Dm7, Am7, Em7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
321,"(Em7, Dm7, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
322,"(C, G, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
323,"(G, Dm, G)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in tqdm(range(1 , len(very_slow) + 1)) :

  very_slow_tf_idf_score_n3.iloc[: , i] = very_slow_tf_idf_df_n3.iloc[:,i]*very_slow_tf_idf_df_n3.loc[:,'IDF-Score']

very_slow_tf_idf_score_n3

100%|██████████| 1943/1943 [00:03<00:00, 646.17it/s] 


,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942
0,"(soc, Am, Em)",0.440877,0.440877,0.0,0.0,0.0,0.0,0.0,0.330592,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,"(Am, Em, F)",0.888816,0.888816,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,"(Em, F, Am)",0.647314,0.647314,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,"(F, Am, Em)",0.647314,0.647314,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,"(Em, F, eoc)",0.400331,0.400331,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,"(Dm7, Am7, Em7)",0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
321,"(Em7, Dm7, eoc)",0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
322,"(C, G, Dm)",0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.511411
323,"(G, Dm, G)",0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.511411


In [ ]:
very_slow_tf_idf_boc_n3 = pd.DataFrame(very_slow_tf_idf_score_n3.T)

very_slow_tf_idf_boc_n3

,0,1,2,3,4,5,6,7,8,9,...,315,316,317,318,319,320,321,322,323,324
chord_prog_token_n3,"(soc, Am, Em)","(Am, Em, F)","(Em, F, Am)","(F, Am, Em)","(Em, F, eoc)","(soc, Am, Dm)","(Am, Dm, E)","(Dm, E, Am)","(E, Am, eoc)","(soc, C, Dm7)",...,"(Fmaj7, C, G)","(C, G, Am7)","(G, Am7, Fmaj7)","(Am7, Em7, Dm7)","(Em7, Dm7, Am7)","(Dm7, Am7, Em7)","(Em7, Dm7, eoc)","(C, G, Dm)","(G, Dm, G)","(Dm, G, Am)"
0,0.440877,0.888816,0.647314,0.647314,0.400331,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.440877,0.888816,0.647314,0.647314,0.400331,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.865279,0.865279,0.865279,0.603233,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.865279,0.865279,0.865279,0.603233,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.490638,0.745319,0.745319,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tf_idf_boc_list_n3 = []

for i in tqdm(range(1, len(very_slow_tf_idf_boc_n3))) : 

  tf_idf_boc_list_n3.append(list(very_slow_tf_idf_boc_n3.iloc[i , :]))

100%|██████████| 1943/1943 [00:00<00:00, 7748.29it/s]


In [ ]:
very_slow_tf_idf_boc_n3 = pd.DataFrame({'id' : very_slow['id']})

very_slow_tf_idf_boc_n3.insert(1 , 'TF-IDF Bag of Chords' , tf_idf_boc_list_n3)

very_slow_tf_idf_boc_n3

,id,TF-IDF Bag of Chords
0,commu00005,"[0.44087728519551317, 0.8888156665239723, 0.64..."
1,commu00011,"[0.44087728519551317, 0.8888156665239723, 0.64..."
2,commu00014,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.8652786463010322, ..."
3,commu00015,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.8652786463010322, ..."
4,commu00020,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
1938,commu10843,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1939,commu10883,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1940,commu10899,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1941,commu11110,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
very_slow_tf_idf_boc_n3.to_csv('TF-IDF_Bag_of_Chords_very_slow_n3.csv')

### b) Slow (61 < bpm <= 90)

#### ㄱ) Chord Progression Token 구하기

In [ ]:
slow.head()

,audio_key,chord_progressions,chord_list,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, G7, C, Am, F, G, C, eoc]",mid_low,8,80,slow,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002
1,cmajor,"[Fmaj7, Fmaj7, Fmaj7, Fmaj7, Fmaj7, Fmaj7, Fma...","[soc, Fmaj7, Em7, Dm7, Cmaj7, eoc]",mid,4,65,slow,newage,pad,string_ensemble,standard,4/4,53,60,train,commu00023
2,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, G7, C, Am, F, G, C, eoc]",mid_high,8,80,slow,newage,main_melody,acoustic_piano,standard,4/4,23,30,train,commu00024
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Em, Em, Em, E...","[soc, Am, Em, F, Em, Am, Em, F, E, eoc]",mid_low,8,70,slow,newage,accompaniment,acoustic_piano,standard,4/4,21,24,train,commu00036
4,cmajor,"[C, C, C, C, C, C, C, C, F, F, F, F, G, G, G, ...","[soc, C, F, G, C, F, Em, C, F, G, C, F, Em, eoc]",mid_high,8,80,slow,newage,main_melody,acoustic_piano,standard,4/4,21,22,train,commu00041


In [ ]:
total_chord_prog_n3 = []

for i in tqdm(range(len(slow))) : 

  target_list = slow.loc[i , 'chord_list'] 

  chord_prog_list = []

  for j in range(len(target_list)-2) :

    chord_prog = []

    chord_prog.append(target_list[j])
    chord_prog.append(target_list[j+1])
    chord_prog.append(target_list[j+2])

    chord_prog = tuple(chord_prog)

    chord_prog_list.append(chord_prog)

  total_chord_prog_n3.append(chord_prog_list)

total_chord_prog_n3

In [ ]:
slow.insert(3 , 'chord_prog_token_n3' , total_chord_prog_n3)

slow.head()

,audio_key,chord_progressions,chord_list,chord_prog_token_n3,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, G7, C, Am, F, G, C, eoc]","[(soc, C, Dm), (C, Dm, G7), (Dm, G7, C), (G7, ...",mid_low,8,80,slow,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002
1,cmajor,"[Fmaj7, Fmaj7, Fmaj7, Fmaj7, Fmaj7, Fmaj7, Fma...","[soc, Fmaj7, Em7, Dm7, Cmaj7, eoc]","[(soc, Fmaj7, Em7), (Fmaj7, Em7, Dm7), (Em7, D...",mid,4,65,slow,newage,pad,string_ensemble,standard,4/4,53,60,train,commu00023
2,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, G7, C, Am, F, G, C, eoc]","[(soc, C, Dm), (C, Dm, G7), (Dm, G7, C), (G7, ...",mid_high,8,80,slow,newage,main_melody,acoustic_piano,standard,4/4,23,30,train,commu00024
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Em, Em, Em, E...","[soc, Am, Em, F, Em, Am, Em, F, E, eoc]","[(soc, Am, Em), (Am, Em, F), (Em, F, Em), (F, ...",mid_low,8,70,slow,newage,accompaniment,acoustic_piano,standard,4/4,21,24,train,commu00036
4,cmajor,"[C, C, C, C, C, C, C, C, F, F, F, F, G, G, G, ...","[soc, C, F, G, C, F, Em, C, F, G, C, F, Em, eoc]","[(soc, C, F), (C, F, G), (F, G, C), (G, C, F),...",mid_high,8,80,slow,newage,main_melody,acoustic_piano,standard,4/4,21,22,train,commu00041


#### ㄴ) TF-IDF 구하기

In [ ]:
prog_token_n3 = []

for i in tqdm(range(len(slow))) :

  for chord_token in slow.loc[i , 'chord_prog_token_n3'] :

    if chord_token not in prog_token_n3 :

      prog_token_n3.append(chord_token)

100%|██████████| 4870/4870 [00:00<00:00, 6955.12it/s]


In [ ]:
slow_tf_idf_df_n3 = pd.DataFrame({'chord_prog_token_n3' : prog_token_n3})

slow_tf_idf_df_n3

,chord_prog_token_n3
0,"(soc, C, Dm)"
1,"(C, Dm, G7)"
2,"(Dm, G7, C)"
3,"(G7, C, Am)"
4,"(C, Am, F)"
...,...
493,"(Gm, G#maj7, eoc)"
494,"(soc, Am, Dm)"
495,"(Am, Dm, Am)"
496,"(Dm, Am, Dm)"


In [ ]:
slow_tf_idf_df_n3 = pd.concat([slow_tf_idf_df_n3 , pd.DataFrame(np.repeat(0 , len(slow_tf_idf_df_n3)*len(slow)).reshape(len(slow_tf_idf_df_n3) , len(slow)))] , axis = 1)

slow_tf_idf_df_n3.insert(len(slow) + 1 , 'IDF-Count' , np.repeat(0 , len(slow_tf_idf_df_n3)))

slow_tf_idf_df_n3

,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,4861,4862,4863,4864,4865,4866,4867,4868,4869,IDF-Count
0,"(soc, C, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(C, Dm, G7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(Dm, G7, C)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(G7, C, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(C, Am, F)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,"(Gm, G#maj7, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
494,"(soc, Am, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
495,"(Am, Dm, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,"(Dm, Am, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for chord_prog_token in tqdm(prog_token_n3) :

  idf_count = 0

  for i in range(len(slow)) : 

    target_list = slow.loc[i , 'chord_prog_token_n3']

    if chord_prog_token not in target_list :

      tf_score = 0

    else :

      idf_count += 1

      each_chord_prog_token_len = len(target_list)

      count = target_list.count(chord_prog_token)

      tf_score = count/each_chord_prog_token_len

      tf_score = round(tf_score , 4)

    slow_tf_idf_df_n3.loc[slow_tf_idf_df_n3['chord_prog_token_n3'] == chord_prog_token , i] = tf_score

  slow_tf_idf_df_n3.loc[slow_tf_idf_df_n3['chord_prog_token_n3'] == chord_prog_token , 'IDF-Count'] = idf_count

100%|██████████| 498/498 [33:46<00:00,  4.07s/it]


In [ ]:
slow_tf_idf_df_n3

,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,4861,4862,4863,4864,4865,4866,4867,4868,4869,IDF-Count
0,"(soc, C, Dm)",0.125,0.0,0.125,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0,0,0.0,0,0,0.0,43
1,"(C, Dm, G7)",0.125,0.0,0.125,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0,0,0.0,0,0,0.0,5
2,"(Dm, G7, C)",0.125,0.0,0.125,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0,0,0.0,0,0,0.0,5
3,"(G7, C, Am)",0.125,0.0,0.125,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0,0,0.0,0,0,0.0,5
4,"(C, Am, F)",0.125,0.0,0.125,0.0,0.0,0.0,0.0,0.125,0.0,...,0.0,0.0,0.0,0,0,0.0,0,0,0.0,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,"(Gm, G#maj7, eoc)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0,0,0.0,0,0,0.0,2
494,"(soc, Am, Dm)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.2,0.0,0,0,0.0,0,0,0.0,4
495,"(Am, Dm, Am)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.4,0.0,0,0,0.0,0,0,0.0,4
496,"(Dm, Am, Dm)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.2,0.0,0,0,0.0,0,0,0.0,4


In [ ]:
import math

idf_score_list_n3 = []

for i in tqdm(range(len(slow_tf_idf_df_n3))) : 

  idf_score = math.log(len(slow)/(1 + slow_tf_idf_df_n3.loc[i , 'IDF-Count']))

  idf_score_list_n3.append(idf_score)

slow_tf_idf_df_n3.insert(len(slow) + 2 , 'IDF-Score' , idf_score_list_n3)

slow_tf_idf_df_n3

100%|██████████| 498/498 [00:00<00:00, 26038.59it/s]
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,4862,4863,4864,4865,4866,4867,4868,4869,IDF-Count,IDF-Score
0,"(soc, C, Dm)",0.125,0.0,0.125,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0,0,0.0,0,0,0.0,43,4.706660
1,"(C, Dm, G7)",0.125,0.0,0.125,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0,0,0.0,0,0,0.0,5,6.699090
2,"(Dm, G7, C)",0.125,0.0,0.125,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0,0,0.0,0,0,0.0,5,6.699090
3,"(G7, C, Am)",0.125,0.0,0.125,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0,0,0.0,0,0,0.0,5,6.699090
4,"(C, Am, F)",0.125,0.0,0.125,0.0,0.0,0.0,0.0,0.125,0.0,...,0.0,0.0,0,0,0.0,0,0,0.0,86,4.024941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,"(Gm, G#maj7, eoc)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0,0,0.0,0,0,0.0,2,7.392237
494,"(soc, Am, Dm)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,...,0.2,0.0,0,0,0.0,0,0,0.0,4,6.881411
495,"(Am, Dm, Am)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,...,0.4,0.0,0,0,0.0,0,0,0.0,4,6.881411
496,"(Dm, Am, Dm)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,...,0.2,0.0,0,0,0.0,0,0,0.0,4,6.881411


In [ ]:
slow_tf_idf_df_n3.to_csv('TF-IDF_slow_n3.csv')

#### ㄷ) TF-IDF Bag of Chord 만들기

In [ ]:
slow_tf_idf_score_n3 = pd.DataFrame({'chord_prog_token_n3' : prog_token_n3})

slow_tf_idf_score_n3 = pd.concat([slow_tf_idf_score_n3 , pd.DataFrame(np.repeat(0 , len(slow_tf_idf_score_n3)*len(slow)).reshape(len(slow_tf_idf_score_n3) , len(slow)))] , axis = 1)
                       
slow_tf_idf_score_n3

,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,4860,4861,4862,4863,4864,4865,4866,4867,4868,4869
0,"(soc, C, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(C, Dm, G7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(Dm, G7, C)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(G7, C, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(C, Am, F)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,"(Gm, G#maj7, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
494,"(soc, Am, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
495,"(Am, Dm, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,"(Dm, Am, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in tqdm(range(1 , len(slow) + 1)) :

  slow_tf_idf_score_n3.iloc[: , i] = slow_tf_idf_df_n3.iloc[:,i]*slow_tf_idf_df_n3.loc[:,'IDF-Score']

slow_tf_idf_score_n3

100%|██████████| 4870/4870 [00:16<00:00, 288.44it/s] 


,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,4860,4861,4862,4863,4864,4865,4866,4867,4868,4869
0,"(soc, C, Dm)",0.588332,0.0,0.588332,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"(C, Dm, G7)",0.837386,0.0,0.837386,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"(Dm, G7, C)",0.837386,0.0,0.837386,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"(G7, C, Am)",0.837386,0.0,0.837386,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"(C, Am, F)",0.503118,0.0,0.503118,0.0,0.0,0.0,0.0,0.503118,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,"(Gm, G#maj7, eoc)",0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
494,"(soc, Am, Dm)",0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,1.376282,0.0,0.0,0.0,0.0,0.0,0.0,0.0
495,"(Am, Dm, Am)",0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,2.752565,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,"(Dm, Am, Dm)",0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,1.376282,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
slow_tf_idf_boc_n3 = pd.DataFrame(slow_tf_idf_score_n3.T)

slow_tf_idf_boc_n3

,0,1,2,3,4,5,6,7,8,9,...,488,489,490,491,492,493,494,495,496,497
chord_prog_token_n3,"(soc, C, Dm)","(C, Dm, G7)","(Dm, G7, C)","(G7, C, Am)","(C, Am, F)","(Am, F, G)","(F, G, C)","(G, C, eoc)","(soc, Fmaj7, Em7)","(Fmaj7, Em7, Dm7)",...,"(soc, Am, eoc)","(soc, Cm, Gm)","(Cm, Gm, G#maj7)","(Gm, G#maj7, Cm)","(G#maj7, Cm, Gm)","(Gm, G#maj7, eoc)","(soc, Am, Dm)","(Am, Dm, Am)","(Dm, Am, Dm)","(Dm, Am, eoc)"
0,0.588332,0.837386,0.837386,0.837386,0.503118,0.707204,0.562733,0.541496,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.015008,0.956853,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.588332,0.837386,0.837386,0.837386,0.503118,0.707204,0.562733,0.541496,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.851792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4866,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.851792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.851792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tf_idf_boc_list_n3 = []

for i in tqdm(range(1, len(slow_tf_idf_boc_n3))) : 

  tf_idf_boc_list_n3.append(list(slow_tf_idf_boc_n3.iloc[i , :]))

100%|██████████| 4870/4870 [00:00<00:00, 6355.92it/s]


In [ ]:
slow_tf_idf_boc_n3 = pd.DataFrame({'id' : slow['id']})

slow_tf_idf_boc_n3.insert(1 , 'TF-IDF Bag of Chords' , tf_idf_boc_list_n3)

slow_tf_idf_boc_n3

,id,TF-IDF Bag of Chords
0,commu00002,"[0.5883324477697968, 0.8373862183560725, 0.837..."
1,commu00023,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.015..."
2,commu00024,"[0.5883324477697968, 0.8373862183560725, 0.837..."
3,commu00036,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,commu00041,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7504609237577..."
...,...,...
4865,commu11125,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4866,commu11128,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4867,commu11137,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4868,commu11138,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
slow_tf_idf_boc_n3.to_csv('TF-IDF_Bag_of_Chords_slow_n3.csv')

### C) Fast (90 < bpm <= 120)

#### ㄱ) Chord Progression Token 구하기

In [ ]:
fast.head()

,audio_key,chord_progressions,chord_list,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, C, C, C, C, C...","[soc, Am, C, G, Dm, Am, C, G, D, eoc]",mid,8,120,fast,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001
1,cmajor,"[C, C, C, C, C, C, C, C, Gm, Gm, Gm, Gm, Gm, G...","[soc, C, Gm, C, Gm, Dm, A#, F, G, eoc]",mid,8,110,fast,cinematic,pad,choir,standard,4/4,45,46,train,commu00004
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, C, C, C, C, C...","[soc, Am, C, G, D, Am, C, G, D, eoc]",mid_high,8,120,fast,cinematic,riff,string_ensemble,standard,4/4,121,122,train,commu00006
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, D, D, D, D, D...","[soc, Am, D, Am, D, Am, D, E, eoc]",mid_low,8,110,fast,cinematic,accompaniment,string_ensemble,standard,4/4,87,96,train,commu00007
4,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, Am, Gsus4, eoc]",mid,4,120,fast,cinematic,sub_melody,brass_ensemble,standard,4/4,68,92,train,commu00008


In [ ]:
total_chord_prog_n3 = []

for i in tqdm(range(len(fast))) : 

  target_list = fast.loc[i , 'chord_list'] 

  chord_prog_list = []

  for j in range(len(target_list)-2) :

    chord_prog = []

    chord_prog.append(target_list[j])
    chord_prog.append(target_list[j+1])
    chord_prog.append(target_list[j+2])

    chord_prog = tuple(chord_prog)

    chord_prog_list.append(chord_prog)

  total_chord_prog_n3.append(chord_prog_list)

total_chord_prog_n3

In [ ]:
fast.insert(3 , 'chord_prog_token_n3' , total_chord_prog_n3)

fast.head()

,audio_key,chord_progressions,chord_list,chord_prog_token_n3,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, C, C, C, C, C...","[soc, Am, C, G, Dm, Am, C, G, D, eoc]","[(soc, Am, C), (Am, C, G), (C, G, Dm), (G, Dm,...",mid,8,120,fast,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001
1,cmajor,"[C, C, C, C, C, C, C, C, Gm, Gm, Gm, Gm, Gm, G...","[soc, C, Gm, C, Gm, Dm, A#, F, G, eoc]","[(soc, C, Gm), (C, Gm, C), (Gm, C, Gm), (C, Gm...",mid,8,110,fast,cinematic,pad,choir,standard,4/4,45,46,train,commu00004
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, C, C, C, C, C...","[soc, Am, C, G, D, Am, C, G, D, eoc]","[(soc, Am, C), (Am, C, G), (C, G, D), (G, D, A...",mid_high,8,120,fast,cinematic,riff,string_ensemble,standard,4/4,121,122,train,commu00006
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, D, D, D, D, D...","[soc, Am, D, Am, D, Am, D, E, eoc]","[(soc, Am, D), (Am, D, Am), (D, Am, D), (Am, D...",mid_low,8,110,fast,cinematic,accompaniment,string_ensemble,standard,4/4,87,96,train,commu00007
4,cmajor,"[C, C, C, C, C, C, C, C, Dm, Dm, Dm, Dm, Dm, D...","[soc, C, Dm, Am, Gsus4, eoc]","[(soc, C, Dm), (C, Dm, Am), (Dm, Am, Gsus4), (...",mid,4,120,fast,cinematic,sub_melody,brass_ensemble,standard,4/4,68,92,train,commu00008


#### ㄴ) TF-IDF 구하기

In [ ]:
prog_token_n3 = []

for i in tqdm(range(len(fast))) :

  for chord_token in fast.loc[i , 'chord_prog_token_n3'] :

    if chord_token not in prog_token_n3 :

      prog_token_n3.append(chord_token)

100%|██████████| 2442/2442 [00:00<00:00, 20863.40it/s]


In [ ]:
fast_tf_idf_df_n3 = pd.DataFrame({'chord_prog_token_n3' : prog_token_n3})

fast_tf_idf_df_n3

,chord_prog_token_n3
0,"(soc, Am, C)"
1,"(Am, C, G)"
2,"(C, G, Dm)"
3,"(G, Dm, Am)"
4,"(Dm, Am, C)"
...,...
291,"(F, Dm, Am)"
292,"(Dm, Am, F)"
293,"(F, Dm, eoc)"
294,"(Fmaj7, Cmaj7, Fmaj7)"


In [ ]:
fast_tf_idf_df_n3 = pd.concat([fast_tf_idf_df_n3 , pd.DataFrame(np.repeat(0 , len(fast_tf_idf_df_n3)*len(fast)).reshape(len(fast_tf_idf_df_n3) , len(fast)))] , axis = 1)

fast_tf_idf_df_n3.insert(len(fast) + 1 , 'IDF-Count' , np.repeat(0 , len(fast_tf_idf_df_n3)))

fast_tf_idf_df_n3                                                      

,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,2433,2434,2435,2436,2437,2438,2439,2440,2441,IDF-Count
0,"(soc, Am, C)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(Am, C, G)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(C, G, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(G, Dm, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(Dm, Am, C)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,"(F, Dm, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
292,"(Dm, Am, F)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
293,"(F, Dm, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,"(Fmaj7, Cmaj7, Fmaj7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for chord_prog_token in tqdm(prog_token_n3) :

  idf_count = 0

  for i in range(len(fast)) : 

    target_list = fast.loc[i , 'chord_prog_token_n3']

    if chord_prog_token not in target_list :

      tf_score = 0

    else :

      idf_count += 1

      each_chord_prog_token_len = len(target_list)

      count = target_list.count(chord_prog_token)

      tf_score = count/each_chord_prog_token_len

      tf_score = round(tf_score , 4)

    fast_tf_idf_df_n3.loc[fast_tf_idf_df_n3['chord_prog_token_n3'] == chord_prog_token , i] = tf_score

  fast_tf_idf_df_n3.loc[fast_tf_idf_df_n3['chord_prog_token_n3'] == chord_prog_token , 'IDF-Count'] = idf_count

100%|██████████| 296/296 [06:39<00:00,  1.35s/it]


In [ ]:
fast_tf_idf_df_n3

,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,2433,2434,2435,2436,2437,2438,2439,2440,2441,IDF-Count
0,"(soc, Am, C)",0.125,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,186
1,"(Am, C, G)",0.250,0.0,0.250,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,135
2,"(C, G, Dm)",0.125,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28
3,"(G, Dm, Am)",0.125,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28
4,"(Dm, Am, C)",0.125,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,"(F, Dm, Am)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
292,"(Dm, Am, F)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
293,"(F, Dm, eoc)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
294,"(Fmaj7, Cmaj7, Fmaj7)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [ ]:
import math

idf_score_list_n3 = []

for i in tqdm(range(len(fast_tf_idf_df_n3))) : 

  idf_score = math.log(len(fast)/(1 + fast_tf_idf_df_n3.loc[i , 'IDF-Count']))

  idf_score_list_n3.append(idf_score)

fast_tf_idf_df_n3.insert(len(fast) + 2 , 'IDF-Score' , idf_score_list_n3)

fast_tf_idf_df_n3

100%|██████████| 296/296 [00:00<00:00, 47735.85it/s]
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,2434,2435,2436,2437,2438,2439,2440,2441,IDF-Count,IDF-Score
0,"(soc, Am, C)",0.125,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,186,2.569464
1,"(Am, C, G)",0.250,0.0,0.250,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,135,2.887918
2,"(C, G, Dm)",0.125,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28,4.433277
3,"(G, Dm, Am)",0.125,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28,4.433277
4,"(Dm, Am, C)",0.125,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28,4.433277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,"(F, Dm, Am)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,6.701960
292,"(Dm, Am, F)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,6.701960
293,"(F, Dm, eoc)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,6.701960
294,"(Fmaj7, Cmaj7, Fmaj7)",0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,6.701960


In [ ]:
fast_tf_idf_df_n3.to_csv('TF-IDF_fast_n3.csv')

#### ㄷ) TF-IDF Bag of Chord 만들기

In [ ]:
fast_tf_idf_score_n3 = pd.DataFrame({'chord_prog_token_n3' : prog_token_n3})

fast_tf_idf_score_n3 = pd.concat([fast_tf_idf_score_n3 , pd.DataFrame(np.repeat(0 , len(fast_tf_idf_score_n3)*len(fast)).reshape(len(fast_tf_idf_score_n3) , len(fast)))] , axis = 1)
                       
fast_tf_idf_score_n3

,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,2432,2433,2434,2435,2436,2437,2438,2439,2440,2441
0,"(soc, Am, C)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(Am, C, G)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(C, G, Dm)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(G, Dm, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(Dm, Am, C)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,"(F, Dm, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
292,"(Dm, Am, F)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
293,"(F, Dm, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,"(Fmaj7, Cmaj7, Fmaj7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in tqdm(range(1 , len(fast) + 1)) :

  fast_tf_idf_score_n3.iloc[: , i] = fast_tf_idf_df_n3.iloc[:,i]*fast_tf_idf_df_n3.loc[:,'IDF-Score']

fast_tf_idf_score_n3

100%|██████████| 2442/2442 [00:04<00:00, 596.06it/s] 


,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,2432,2433,2434,2435,2436,2437,2438,2439,2440,2441
0,"(soc, Am, C)",0.321183,0.0,0.321183,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"(Am, C, G)",0.721979,0.0,0.721979,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"(C, G, Dm)",0.554160,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"(G, Dm, Am)",0.554160,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"(Dm, Am, C)",0.554160,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,"(F, Dm, Am)",0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
292,"(Dm, Am, F)",0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,"(F, Dm, eoc)",0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
294,"(Fmaj7, Cmaj7, Fmaj7)",0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
fast_tf_idf_boc_n3 = pd.DataFrame(fast_tf_idf_score_n3.T)

fast_tf_idf_boc_n3

,0,1,2,3,4,5,6,7,8,9,...,286,287,288,289,290,291,292,293,294,295
chord_prog_token_n3,"(soc, Am, C)","(Am, C, G)","(C, G, Dm)","(G, Dm, Am)","(Dm, Am, C)","(C, G, D)","(G, D, eoc)","(soc, C, Gm)","(C, Gm, C)","(Gm, C, Gm)",...,"(Fmaj7, Gsus4, G)","(soc, Cmaj7, Dm7)","(Cmaj7, Dm7, Em7)","(Dm7, Em7, Fmaj7)","(Em7, Fmaj7, eoc)","(F, Dm, Am)","(Dm, Am, F)","(F, Dm, eoc)","(Fmaj7, Cmaj7, Fmaj7)","(Cmaj7, Fmaj7, eoc)"
0,0.321183,0.721979,0.55416,0.55416,0.55416,0.46969,0.428891,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.465379,0.465379,0.465379,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.321183,0.721979,0.0,0.0,0.0,0.93938,0.428891,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tf_idf_boc_list_n3 = []

for i in tqdm(range(1, len(fast_tf_idf_boc_n3))) : 

  tf_idf_boc_list_n3.append(list(fast_tf_idf_boc_n3.iloc[i , :]))

100%|██████████| 2442/2442 [00:00<00:00, 6946.62it/s]


In [ ]:
fast_tf_idf_boc_n3 = pd.DataFrame({'id' : fast['id']})

fast_tf_idf_boc_n3.insert(1 , 'TF-IDF Bag of Chords' , tf_idf_boc_list_n3)

fast_tf_idf_boc_n3

,id,TF-IDF Bag of Chords
0,commu00001,"[0.3211830047270079, 0.7219794422336495, 0.554..."
1,commu00004,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.46537940..."
2,commu00006,"[0.3211830047270079, 0.7219794422336495, 0.0, ..."
3,commu00007,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,commu00008,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
2437,commu11133,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2438,commu11135,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2439,commu11136,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2440,commu11140,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
fast_tf_idf_boc_n3.to_csv('TF-IDF_Bag_of_Chords_fast_n3.csv')

### d) Very_fast (bpm > 120)

#### ㄱ) Chord Progression Token 구하기

In [ ]:
very_fast.head()

,audio_key,chord_progressions,chord_list,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]",mid_high,8,150,very_fast,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003
1,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]",mid_low,8,150,very_fast,cinematic,accompaniment,string_ensemble,standard,4/4,113,122,train,commu00010
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, E, F, G#dim, eoc]",mid_high,8,150,very_fast,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00012
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, F, F, F, F, F...","[soc, Am, F, C, G, eoc]",mid,4,130,very_fast,newage,pad,acoustic_piano-2,standard,4/4,55,56,train,commu00013
4,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]",mid,8,150,very_fast,cinematic,main_melody,brass_ensemble,standard,4/4,83,91,train,commu00019


In [ ]:
total_chord_prog_n3 = []

for i in tqdm(range(len(very_fast))) : 

  target_list = very_fast.loc[i , 'chord_list'] 

  chord_prog_list = []

  for j in range(len(target_list)-2) :

    chord_prog = []

    chord_prog.append(target_list[j])
    chord_prog.append(target_list[j+1])
    chord_prog.append(target_list[j+2])

    chord_prog = tuple(chord_prog)

    chord_prog_list.append(chord_prog)

  total_chord_prog_n3.append(chord_prog_list)

total_chord_prog_n3

100%|██████████| 1889/1889 [00:00<00:00, 38473.13it/s]


[[('soc', 'Am', 'F'), ('Am', 'F', 'E+'), ('F', 'E+', 'E'), ('E+', 'E', 'eoc')],
 [('soc', 'Am', 'F'), ('Am', 'F', 'E+'), ('F', 'E+', 'E'), ('E+', 'E', 'eoc')],
 [('soc', 'Am', 'E'),
  ('Am', 'E', 'F'),
  ('E', 'F', 'G#dim'),
  ('F', 'G#dim', 'eoc')],
 [('soc', 'Am', 'F'), ('Am', 'F', 'C'), ('F', 'C', 'G'), ('C', 'G', 'eoc')],
 [('soc', 'Am', 'F'), ('Am', 'F', 'E+'), ('F', 'E+', 'E'), ('E+', 'E', 'eoc')],
 [('soc', 'C', 'G'),
  ('C', 'G', 'Am'),
  ('G', 'Am', 'F'),
  ('Am', 'F', 'C'),
  ('F', 'C', 'Em'),
  ('C', 'Em', 'Am'),
  ('Em', 'Am', 'F'),
  ('Am', 'F', 'eoc')],
 [('soc', 'Am', 'E'),
  ('Am', 'E', 'F'),
  ('E', 'F', 'G#dim'),
  ('F', 'G#dim', 'eoc')],
 [('soc', 'Am', 'G#dim'),
  ('Am', 'G#dim', 'F'),
  ('G#dim', 'F', 'E'),
  ('F', 'E', 'eoc')],
 [('soc', 'Am', 'F'), ('Am', 'F', 'E+'), ('F', 'E+', 'E'), ('E+', 'E', 'eoc')],
 [('soc', 'Am', 'G#dim'),
  ('Am', 'G#dim', 'F'),
  ('G#dim', 'F', 'E'),
  ('F', 'E', 'eoc')],
 [('soc', 'C', 'eoc')],
 [('soc', 'C', 'G'),
  ('C', 'G', 'Am'),


In [ ]:
very_fast.insert(3 , 'chord_prog_token_n3' , total_chord_prog_n3)

very_fast.head()

,audio_key,chord_progressions,chord_list,chord_prog_token_n3,pitch_range,num_measures,bpm,bpm_group,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id
0,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]","[(soc, Am, F), (Am, F, E+), (F, E+, E), (E+, E...",mid_high,8,150,very_fast,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003
1,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]","[(soc, Am, F), (Am, F, E+), (F, E+, E), (E+, E...",mid_low,8,150,very_fast,cinematic,accompaniment,string_ensemble,standard,4/4,113,122,train,commu00010
2,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, E, F, G#dim, eoc]","[(soc, Am, E), (Am, E, F), (E, F, G#dim), (F, ...",mid_high,8,150,very_fast,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00012
3,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, F, F, F, F, F...","[soc, Am, F, C, G, eoc]","[(soc, Am, F), (Am, F, C), (F, C, G), (C, G, e...",mid,4,130,very_fast,newage,pad,acoustic_piano-2,standard,4/4,55,56,train,commu00013
4,aminor,"[Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, Am, A...","[soc, Am, F, E+, E, eoc]","[(soc, Am, F), (Am, F, E+), (F, E+, E), (E+, E...",mid,8,150,very_fast,cinematic,main_melody,brass_ensemble,standard,4/4,83,91,train,commu00019


#### ㄴ) TF-IDF 구하기

In [ ]:
prog_token_n3 = []

for i in tqdm(range(len(very_fast))) :

  for chord_token in very_fast.loc[i , 'chord_prog_token_n3'] :

    if chord_token not in prog_token_n3 :

      prog_token_n3.append(chord_token)

100%|██████████| 1889/1889 [00:00<00:00, 11804.40it/s]


In [ ]:
very_fast_tf_idf_df_n3 = pd.DataFrame({'chord_prog_token_n3' : prog_token_n3})

very_fast_tf_idf_df_n3

,chord_prog_token_n3
0,"(soc, Am, F)"
1,"(Am, F, E+)"
2,"(F, E+, E)"
3,"(E+, E, eoc)"
4,"(soc, Am, E)"
...,...
320,"(Am, Am7, Am)"
321,"(Am7, Am, Am7)"
322,"(Am, Am7, eoc)"
323,"(Am, Dm, E)"


In [ ]:
very_fast_tf_idf_df_n3 = pd.concat([very_fast_tf_idf_df_n3 , pd.DataFrame(np.repeat(0 , len(very_fast_tf_idf_df_n3)*len(very_fast)).reshape(len(very_fast_tf_idf_df_n3) , len(very_fast)))] , axis = 1)

very_fast_tf_idf_df_n3.insert(len(very_fast) + 1 , 'IDF-Count' , np.repeat(0 , len(very_fast_tf_idf_df_n3)))

very_fast_tf_idf_df_n3                                                      

,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,1880,1881,1882,1883,1884,1885,1886,1887,1888,IDF-Count
0,"(soc, Am, F)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(Am, F, E+)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(F, E+, E)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(E+, E, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(soc, Am, E)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,"(Am, Am7, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
321,"(Am7, Am, Am7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
322,"(Am, Am7, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
323,"(Am, Dm, E)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for chord_prog_token in tqdm(prog_token_n3) :

  idf_count = 0

  for i in range(len(very_fast)) : 

    target_list = very_fast.loc[i , 'chord_prog_token_n3']

    if chord_prog_token not in target_list :

      tf_score = 0

    else :

      idf_count += 1

      each_chord_prog_token_len = len(target_list)

      count = target_list.count(chord_prog_token)

      tf_score = count/each_chord_prog_token_len

      tf_score = round(tf_score , 4)

    very_fast_tf_idf_df_n3.loc[very_fast_tf_idf_df_n3['chord_prog_token_n3'] == chord_prog_token , i] = tf_score

  very_fast_tf_idf_df_n3.loc[very_fast_tf_idf_df_n3['chord_prog_token_n3'] == chord_prog_token , 'IDF-Count'] = idf_count

100%|██████████| 325/325 [05:27<00:00,  1.01s/it]


In [ ]:
very_fast_tf_idf_df_n3

,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,1880,1881,1882,1883,1884,1885,1886,1887,1888,IDF-Count
0,"(soc, Am, F)",0.25,0.25,0.00,0.25,0.25,0.0,0.00,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.00,0.0,193
1,"(Am, F, E+)",0.25,0.25,0.00,0.00,0.25,0.0,0.00,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,71
2,"(F, E+, E)",0.25,0.25,0.00,0.00,0.25,0.0,0.00,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,71
3,"(E+, E, eoc)",0.25,0.25,0.00,0.00,0.25,0.0,0.00,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,71
4,"(soc, Am, E)",0.00,0.00,0.25,0.00,0.00,0.0,0.25,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,"(Am, Am7, Am)",0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.25,0.0,9
321,"(Am7, Am, Am7)",0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.25,0.0,9
322,"(Am, Am7, eoc)",0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.25,0.0,9
323,"(Am, Dm, E)",0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,7


In [ ]:
import math

idf_score_list_n3 = []

for i in tqdm(range(len(very_fast_tf_idf_df_n3))) : 

  idf_score = math.log(len(very_fast)/(1 + very_fast_tf_idf_df_n3.loc[i , 'IDF-Count']))

  idf_score_list_n3.append(idf_score)

very_fast_tf_idf_df_n3.insert(len(very_fast) + 2 , 'IDF-Score' , idf_score_list_n3)

very_fast_tf_idf_df_n3

100%|██████████| 325/325 [00:00<00:00, 41475.96it/s]
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,1881,1882,1883,1884,1885,1886,1887,1888,IDF-Count,IDF-Score
0,"(soc, Am, F)",0.25,0.25,0.00,0.25,0.25,0.0,0.00,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.25,0.00,0.0,193,2.275945
1,"(Am, F, E+)",0.25,0.25,0.00,0.00,0.25,0.0,0.00,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,71,3.267137
2,"(F, E+, E)",0.25,0.25,0.00,0.00,0.25,0.0,0.00,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,71,3.267137
3,"(E+, E, eoc)",0.25,0.25,0.00,0.00,0.25,0.0,0.00,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,71,3.267137
4,"(soc, Am, E)",0.00,0.00,0.25,0.00,0.00,0.0,0.25,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,65,3.354148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,"(Am, Am7, Am)",0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.25,0.0,9,5.241218
321,"(Am7, Am, Am7)",0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.25,0.0,9,5.241218
322,"(Am, Am7, eoc)",0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.25,0.0,9,5.241218
323,"(Am, Dm, E)",0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,7,5.464361


In [ ]:
very_fast_tf_idf_df_n3.to_csv('TF-IDF_very_fast_n3.csv')

#### ㄷ) TF-IDF Bag of Chord 만들기

In [ ]:
very_fast_tf_idf_score_n3 = pd.DataFrame({'chord_prog_token_n3' : prog_token_n3})

very_fast_tf_idf_score_n3 = pd.concat([very_fast_tf_idf_score_n3 , pd.DataFrame(np.repeat(0 , len(very_fast_tf_idf_score_n3)*len(very_fast)).reshape(len(very_fast_tf_idf_score_n3) , len(very_fast)))] , axis = 1)
                       
very_fast_tf_idf_score_n3         

,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888
0,"(soc, Am, F)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"(Am, F, E+)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"(F, E+, E)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"(E+, E, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(soc, Am, E)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,"(Am, Am7, Am)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
321,"(Am7, Am, Am7)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
322,"(Am, Am7, eoc)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
323,"(Am, Dm, E)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in tqdm(range(1 , len(very_fast) + 1)) :

  very_fast_tf_idf_score_n3.iloc[: , i] = very_fast_tf_idf_df_n3.iloc[:,i]*very_fast_tf_idf_df_n3.loc[:,'IDF-Score']

very_fast_tf_idf_score_n3

100%|██████████| 1889/1889 [00:01<00:00, 977.28it/s] 


,chord_prog_token_n3,0,1,2,3,4,5,6,7,8,...,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888
0,"(soc, Am, F)",0.568986,0.568986,0.000000,0.568986,0.568986,0.0,0.000000,0.0,0.568986,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.568986,0.000000,0.0
1,"(Am, F, E+)",0.816784,0.816784,0.000000,0.000000,0.816784,0.0,0.000000,0.0,0.816784,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
2,"(F, E+, E)",0.816784,0.816784,0.000000,0.000000,0.816784,0.0,0.000000,0.0,0.816784,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
3,"(E+, E, eoc)",0.816784,0.816784,0.000000,0.000000,0.816784,0.0,0.000000,0.0,0.816784,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
4,"(soc, Am, E)",0.000000,0.000000,0.838537,0.000000,0.000000,0.0,0.838537,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,"(Am, Am7, Am)",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.310304,0.0
321,"(Am7, Am, Am7)",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.310304,0.0
322,"(Am, Am7, eoc)",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.310304,0.0
323,"(Am, Dm, E)",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0


In [ ]:
very_fast_tf_idf_boc_n3 = pd.DataFrame(very_fast_tf_idf_score_n3.T)

very_fast_tf_idf_boc_n3

,0,1,2,3,4,5,6,7,8,9,...,315,316,317,318,319,320,321,322,323,324
chord_prog_token_n3,"(soc, Am, F)","(Am, F, E+)","(F, E+, E)","(E+, E, eoc)","(soc, Am, E)","(Am, E, F)","(E, F, G#dim)","(F, G#dim, eoc)","(Am, F, C)","(F, C, G)",...,"(F, Am, Em)","(Am, Dm, Bdim)","(Dm, Bdim, Dm)","(Bdim, Dm, eoc)","(soc, Am, Am7)","(Am, Am7, Am)","(Am7, Am, Am7)","(Am, Am7, eoc)","(Am, Dm, E)","(Dm, E, Am)"
0,0.568986,0.816784,0.816784,0.816784,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.568986,0.816784,0.816784,0.816784,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.838537,0.838537,0.838537,0.838537,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.568986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.643497,0.732171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.829975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1886,0.568986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1887,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.310304,1.310304,1.310304,1.310304,0.0,0.0


In [ ]:
tf_idf_boc_list_n3 = []

for i in tqdm(range(1, len(very_fast_tf_idf_boc_n3))) : 

  tf_idf_boc_list_n3.append(list(very_fast_tf_idf_boc_n3.iloc[i , :]))

100%|██████████| 1889/1889 [00:00<00:00, 7332.12it/s]


In [ ]:
very_fast_tf_idf_boc_n3 = pd.DataFrame({'id' : very_fast['id']})

very_fast_tf_idf_boc_n3.insert(1 , 'TF-IDF Bag of Chords' , tf_idf_boc_list_n3)

very_fast_tf_idf_boc_n3

,id,TF-IDF Bag of Chords
0,commu00003,"[0.5689861771095454, 0.8167841871213636, 0.816..."
1,commu00010,"[0.5689861771095454, 0.8167841871213636, 0.816..."
2,commu00012,"[0.0, 0.0, 0.0, 0.0, 0.838537031368771, 0.8385..."
3,commu00013,"[0.5689861771095454, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,commu00019,"[0.5689861771095454, 0.8167841871213636, 0.816..."
...,...,...
1884,commu11104,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1885,commu11127,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1886,commu11142,"[0.5689861771095454, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1887,commu11143,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
very_fast_tf_idf_boc_n3.to_csv('TF-IDF_Bag_of_Chords_very_fast_n3.csv')

## 3) BPM 별 각 commu data에 대한 코사인 유사도 계산

### a) Very_slow (bpm <= 60)

In [ ]:
very_slow_tf_idf_boc_n3 = pd.read_csv('TF-IDF_Bag_of_Chords_very_slow_n3.csv')

very_slow_tf_idf_boc_n3.drop('Unnamed: 0' , axis = 1 , inplace = True)

very_slow_tf_idf_boc_n3

,id,TF-IDF Bag of Chords
0,commu00005,"[0.23243654444615658, 0.6877377472371866, 0.52..."
1,commu00011,"[0.23243654444615658, 0.6877377472371866, 0.52..."
2,commu00014,"[0.325313568154173, 0.0, 0.0, 0.0, 0.0, 0.6922..."
3,commu00015,"[0.325313568154173, 0.0, 0.0, 0.0, 0.0, 0.6922..."
4,commu00020,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
1938,commu10843,"[0.8132839203854324, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1939,commu10883,"[0.0, 0.0, 0.0, 0.2326951790003213, 0.15329401..."
1940,commu10899,"[0.0, 0.0, 0.0, 0.2326951790003213, 0.15329401..."
1941,commu11110,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
test = very_slow_tf_idf_boc_n3['TF-IDF Bag of Chords']

for i in range(len(test)) :

  test[i] = test[i].replace("'" , "").replace(" " , "").replace("[" , "").replace("]" , "")
  test[i] = test[i].split(",")

  test[i] = list(map(float , test[i]))

very_slow_tf_idf_boc_n3['TF-IDF Bag of Chords'] = test

In [ ]:
id_ord = list(very_slow_tf_idf_boc_n3['id'])

id_order = CategoricalDtype(categories = id_ord , ordered = True)

very_slow_tf_idf_boc_n3['id'] = very_slow_tf_idf_boc_n3['id'].astype(id_order)

very_slow_tf_idf_boc_n3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1943 entries, 0 to 1942
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    1943 non-null   category
 1   TF-IDF Bag of Chords  1943 non-null   object  
dtypes: category(1), object(1)
memory usage: 98.8+ KB


In [ ]:
very_slow_tf_idf_boc_n3['id']

0       commu00005
1       commu00011
2       commu00014
3       commu00015
4       commu00020
           ...    
1938    commu10843
1939    commu10883
1940    commu10899
1941    commu11110
1942    commu11134
Name: id, Length: 1943, dtype: category
Categories (1943, object): ['commu00005' < 'commu00011' < 'commu00014' < 'commu00015' < ... <
                            'commu10883' < 'commu10899' < 'commu11110' < 'commu11134']

In [ ]:
id = very_slow_tf_idf_boc_n3['id']

id

0       commu00005
1       commu00011
2       commu00014
3       commu00015
4       commu00020
           ...    
1938    commu10843
1939    commu10883
1940    commu10899
1941    commu11110
1942    commu11134
Name: id, Length: 1943, dtype: category
Categories (1943, object): ['commu00005' < 'commu00011' < 'commu00014' < 'commu00015' < ... <
                            'commu10883' < 'commu10899' < 'commu11110' < 'commu11134']

In [ ]:
very_slow_similarity_df_n3 = pd.DataFrame(np.repeat(0 , len(id)**2).reshape(len(id),len(id)))

very_slow_similarity_df_n3

,0,1,2,3,4,5,6,7,8,9,...,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1939,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1941,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
len(very_slow_similarity_df_n3.columns)

1943

In [ ]:
id_list_1 = very_slow_tf_idf_boc_n3['id'][:972]
id_list_2 = very_slow_tf_idf_boc_n3['id'][972:]

print(len(id_list_1))
print(len(id_list_2))

972
971


In [ ]:
very_slow_1 = very_slow_similarity_df_n3.iloc[:972 , :972].copy()
very_slow_2 = very_slow_similarity_df_n3.iloc[:972 , 972:].copy()
very_slow_3 = very_slow_similarity_df_n3.iloc[972: , :972].copy()
very_slow_4 = very_slow_similarity_df_n3.iloc[972: , 972:].copy()

id_list_1 = list(very_slow_tf_idf_boc_n3['id'][:972])
id_list_2 = list(very_slow_tf_idf_boc_n3['id'][972:])

very_slow_1.columns = id_list_1
very_slow_1.insert(0 , 'id' , id_list_1)

very_slow_2.columns = id_list_2
very_slow_2.insert(0 , 'id' , id_list_1)

very_slow_3.columns = id_list_1
very_slow_3.insert(0 , 'id' , id_list_2)

very_slow_4.columns = id_list_2
very_slow_4.insert(0 , 'id' , id_list_2)

In [ ]:
for id_1 in tqdm(id_list_1) :

  for id_2 in id_list_1 :

    if id_1 < id_2 :

      A = very_slow_tf_idf_boc_n3.loc[very_slow_tf_idf_boc_n3['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = very_slow_tf_idf_boc_n3.loc[very_slow_tf_idf_boc_n3['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      very_slow_1.loc[very_slow_1['id'] == id_1 , id_2] = cos_sim(A , B)

very_slow_1

100%|██████████| 972/972 [11:44<00:00,  1.38it/s]


,id,commu00005,commu00011,commu00014,commu00015,commu00020,commu00025,commu00026,commu00044,commu00048,...,commu05647,commu05656,commu05659,commu05661,commu05676,commu05677,commu05682,commu05683,commu05684,commu05687
0,commu00005,0,1,0,0,0,0.0,0,0.090586,0.0,...,0.0,0.0,0.176218,0.0,0.0,0.0,0.176218,0.176218,0.0,0.000000
1,commu00011,0,0,0,0,0,0.0,0,0.090586,0.0,...,0.0,0.0,0.176218,0.0,0.0,0.0,0.176218,0.176218,0.0,0.000000
2,commu00014,0,0,0,1,0,0.0,1,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
3,commu00015,0,0,0,0,0,0.0,1,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
4,commu00020,0,0,0,0,0,1.0,0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,commu05677,0,0,0,0,0,0.0,0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
968,commu05682,0,0,0,0,0,0.0,0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.0,0.074418
969,commu05683,0,0,0,0,0,0.0,0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.074418
970,commu05684,0,0,0,0,0,0.0,0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000


In [ ]:
for id_1 in tqdm(id_list_1) :

  for id_2 in id_list_2 :

    if id_1 < id_2 :

      A = very_slow_tf_idf_boc_n3.loc[very_slow_tf_idf_boc_n3['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = very_slow_tf_idf_boc_n3.loc[very_slow_tf_idf_boc_n3['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      very_slow_2.loc[very_slow_2['id'] == id_1 , id_2] = cos_sim(A , B)

very_slow_2

100%|██████████| 972/972 [23:16<00:00,  1.44s/it]


,id,commu05688,commu05695,commu05699,commu05702,commu05703,commu05714,commu05715,commu05716,commu05717,...,commu10826,commu10829,commu10833,commu10835,commu10841,commu10843,commu10883,commu10899,commu11110,commu11134
0,commu00005,0.000000,0.0,0.000000,0.133071,0.000000,0.000000,0.176218,0.0,0.133071,...,0.000000,0.000000,0.000000,0.0,0.0,0,0.000000,0.000000,0.0,0.0
1,commu00011,0.000000,0.0,0.000000,0.133071,0.000000,0.000000,0.176218,0.0,0.133071,...,0.000000,0.000000,0.000000,0.0,0.0,0,0.000000,0.000000,0.0,0.0
2,commu00014,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.063041,0.000000,0.000000,0.0,0.0,0,0.000000,0.000000,0.0,0.0
3,commu00015,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.063041,0.000000,0.000000,0.0,0.0,0,0.000000,0.000000,0.0,0.0
4,commu00020,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,commu05677,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0,0.213265,0.213265,0.0,0.0
968,commu05682,0.000000,0.0,0.000000,0.000000,0.074418,0.074418,1.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0,0.000000,0.000000,0.0,0.0
969,commu05683,0.000000,0.0,0.000000,0.000000,0.074418,0.074418,1.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0,0.000000,0.000000,0.0,0.0
970,commu05684,0.120275,0.0,0.120275,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.054285,0.054285,0.0,0.0,0,0.000000,0.000000,0.0,0.0


In [ ]:
for id_1 in tqdm(id_list_2) :

  for id_2 in id_list_2 :

    if id_1 < id_2 :

      A = very_slow_tf_idf_boc_n3.loc[very_slow_tf_idf_boc_n3['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = very_slow_tf_idf_boc_n3.loc[very_slow_tf_idf_boc_n3['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      very_slow_4.loc[very_slow_4['id'] == id_1 , id_2] = cos_sim(A , B)

very_slow_4

100%|██████████| 971/971 [11:35<00:00,  1.40it/s]


,id,commu05688,commu05695,commu05699,commu05702,commu05703,commu05714,commu05715,commu05716,commu05717,...,commu10826,commu10829,commu10833,commu10835,commu10841,commu10843,commu10883,commu10899,commu11110,commu11134
972,commu05688,0,0,1,0,0,0.0,0.000000,0,0.0,...,0.0,0.038808,0.038808,0.0,0.0,0,0.0,0.0,0.0,0.0
973,commu05695,0,0,0,0,0,0.0,0.000000,0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.0
974,commu05699,0,0,0,0,0,0.0,0.000000,0,0.0,...,0.0,0.038808,0.038808,0.0,0.0,0,0.0,0.0,0.0,0.0
975,commu05702,0,0,0,0,0,0.0,0.000000,0,1.0,...,0.0,0.000000,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.0
976,commu05703,0,0,0,0,0,1.0,0.074418,0,0.0,...,0.0,0.047078,0.047078,0.0,0.0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938,commu10843,0,0,0,0,0,0.0,0.000000,0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.0
1939,commu10883,0,0,0,0,0,0.0,0.000000,0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0,0.0,1.0,0.0,0.0
1940,commu10899,0,0,0,0,0,0.0,0.000000,0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.0
1941,commu11110,0,0,0,0,0,0.0,0.000000,0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.0


In [ ]:
very_slow_12 = pd.concat([very_slow_1 , very_slow_2] , axis = 1)
very_slow_34 = pd.concat([very_slow_3 , very_slow_4] , axis = 1)

very_slow_similarity_df_n3 = pd.concat([very_slow_12 , very_slow_34])

very_slow_similarity_df_n3

,id,commu00005,commu00011,commu00014,commu00015,commu00020,commu00025,commu00026,commu00044,commu00048,...,commu10826,commu10829,commu10833,commu10835,commu10841,commu10843,commu10883,commu10899,commu11110,commu11134
0,commu00005,0,1,0,0,0,0.0,0,0.090586,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
1,commu00011,0,0,0,0,0,0.0,0,0.090586,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
2,commu00014,0,0,0,1,0,0.0,1,0.000000,0.0,...,0.063041,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
3,commu00015,0,0,0,0,0,0.0,1,0.000000,0.0,...,0.063041,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
4,commu00020,0,0,0,0,0,1.0,0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938,commu10843,0,0,0,0,0,0.0,0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
1939,commu10883,0,0,0,0,0,0.0,0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0
1940,commu10899,0,0,0,0,0,0.0,0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
1941,commu11110,0,0,0,0,0,0.0,0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0


In [ ]:
very_slow_similarity_df_n3.to_csv('cos_sim_very_slow_n3.csv')

### b) Slow (60 < bpm <= 90)

In [ ]:
slow_tf_idf_boc_n3 = pd.read_csv('TF-IDF_Bag_of_Chords_slow_n3.csv')

slow_tf_idf_boc_n3.drop('Unnamed: 0' , axis = 1 , inplace = True)

slow_tf_idf_boc_n3

,id,TF-IDF Bag of Chords
0,commu00002,"[0.5883324477697968, 0.8373862183560725, 0.837..."
1,commu00023,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.015..."
2,commu00024,"[0.5883324477697968, 0.8373862183560725, 0.837..."
3,commu00036,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,commu00041,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7504609237577..."
...,...,...
4865,commu11125,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4866,commu11128,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4867,commu11137,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4868,commu11138,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
test = very_slow_tf_idf_boc_n3['TF-IDF Bag of Chords']

for i in range(len(test)) :

  test[i] = test[i].replace("'" , "").replace(" " , "").replace("[" , "").replace("]" , "")
  test[i] = test[i].split(",")

  test[i] = list(map(float , test[i]))

very_slow_tf_idf_boc_n3['TF-IDF Bag of Chords'] = test

In [ ]:
id_ord = list(slow_tf_idf_boc_n3['id'])

id_order = CategoricalDtype(categories = id_ord , ordered = True)

slow_tf_idf_boc_n3['id'] = slow_tf_idf_boc_n3['id'].astype(id_order)

slow_tf_idf_boc_n3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4870 entries, 0 to 4869
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    4870 non-null   category
 1   TF-IDF Bag of Chords  4870 non-null   object  
dtypes: category(1), object(1)
memory usage: 214.8+ KB


In [ ]:
id = slow_tf_idf_boc_n3['id']

id

0       commu00002
1       commu00023
2       commu00024
3       commu00036
4       commu00041
           ...    
4865    commu11125
4866    commu11128
4867    commu11137
4868    commu11138
4869    commu11139
Name: id, Length: 4870, dtype: category
Categories (4870, object): ['commu00002' < 'commu00023' < 'commu00024' < 'commu00036' < ... <
                            'commu11128' < 'commu11137' < 'commu11138' < 'commu11139']

In [ ]:
slow_similarity_df_n3 = pd.DataFrame(np.repeat(0 , len(id)**2).reshape(len(id),len(id)))

slow_similarity_df_n3

,0,1,2,3,4,5,6,7,8,9,...,4860,4861,4862,4863,4864,4865,4866,4867,4868,4869
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4866,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4867,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4868,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
id_list_1 = slow_tf_idf_boc_n3['id'][:2435]
id_list_2 = slow_tf_idf_boc_n3['id'][2435:]

In [ ]:
slow_1 = slow_similarity_df_n3.iloc[:2435 , :2435].copy()
slow_2 = slow_similarity_df_n3.iloc[:2435 , :2435].copy()
slow_3 = slow_similarity_df_n3.iloc[:2435 , :2435].copy()
slow_4 = slow_similarity_df_n3.iloc[:2435 , :2435].copy()

id_list_1 = list(slow_tf_idf_boc_n3['id'][:2435])
id_list_2 = list(slow_tf_idf_boc_n3['id'][2435:])

slow_1.columns = id_list_1
slow_1.insert(0 , 'id' , id_list_1)

slow_2.columns = id_list_2
slow_2.insert(0 , 'id' , id_list_1)

slow_3.columns = id_list_1
slow_3.insert(0 , 'id' , id_list_2)

slow_4.columns = id_list_2
slow_4.insert(0 , 'id' , id_list_2)

In [ ]:
for id_1 in tqdm(id_list_1) :

  for id_2 in id_list_1 :

    if id_1 < id_2 :

      A = slow_tf_idf_boc_n3.loc[slow_tf_idf_boc_n3['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = slow_tf_idf_boc_n3.loc[slow_tf_idf_boc_n3['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      slow_1.loc[slow_1['id'] == id_1 , id_2] = cos_sim(A , B)

slow_1

100%|██████████| 2435/2435 [1:26:37<00:00,  2.13s/it]


,id,commu00002,commu00023,commu00024,commu00036,commu00041,commu00042,commu00043,commu00046,commu00054,...,commu05498,commu05501,commu05502,commu05503,commu05504,commu05505,commu05506,commu05507,commu05508,commu05514
0,commu00002,0,0,1.0,0,0.118854,0,0.000000,0.248915,0,...,0.000000,0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
1,commu00023,0,0,0.0,0,0.000000,0,0.000000,0.000000,0,...,0.000000,0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
2,commu00024,0,0,0.0,0,0.118854,0,0.000000,0.248915,0,...,0.000000,0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
3,commu00036,0,0,0.0,0,0.000000,0,0.000000,0.000000,0,...,0.097267,0,0.097267,0.0,0.0,0.211516,1.000000,0.0,0.040421,0.0
4,commu00041,0,0,0.0,0,0.000000,0,0.154447,0.000000,0,...,0.000000,0,0.000000,0.0,0.0,0.063784,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430,commu05505,0,0,0.0,0,0.000000,0,0.000000,0.000000,0,...,0.000000,0,0.000000,0.0,0.0,0.000000,0.211516,0.0,0.047145,0.0
2431,commu05506,0,0,0.0,0,0.000000,0,0.000000,0.000000,0,...,0.000000,0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.040421,0.0
2432,commu05507,0,0,0.0,0,0.000000,0,0.000000,0.000000,0,...,0.000000,0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
2433,commu05508,0,0,0.0,0,0.000000,0,0.000000,0.000000,0,...,0.000000,0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0


In [ ]:
slow_1.to_csv('slow_1.csv')

In [ ]:
for id_1 in tqdm(id_list_1) :

  for id_2 in id_list_2 :

    if id_1 < id_2 :

      A = slow_tf_idf_boc_n3.loc[slow_tf_idf_boc_n3['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = slow_tf_idf_boc_n3.loc[slow_tf_idf_boc_n3['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      slow_2.loc[slow_2['id'] == id_1 , id_2] = cos_sim(A , B)

slow_2

100%|██████████| 2435/2435 [2:58:16<00:00,  4.39s/it]


,id,commu05516,commu05517,commu05519,commu05520,commu05521,commu05523,commu05525,commu05526,commu05528,...,commu11114,commu11116,commu11117,commu11120,commu11124,commu11125,commu11128,commu11137,commu11138,commu11139
0,commu00002,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0,0.0,0,0.0,0,0,0.000000,0,0,0.000000
1,commu00023,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0,0.0,0,0.0,0,0,0.000000,0,0,0.000000
2,commu00024,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0,0.0,0,0.0,0,0,0.000000,0,0,0.000000
3,commu00036,1.000000,0.039367,0.211516,0.097267,0.040421,0.0,0.211516,0.044020,0.0,...,0,0.0,0,0.0,0,0,0.000000,0,0,0.000000
4,commu00041,0.000000,0.000000,0.063784,0.000000,0.000000,0.0,0.063784,0.000000,0.0,...,0,0.0,0,0.0,0,0,0.000000,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430,commu05505,0.211516,0.045916,1.000000,0.000000,0.047145,0.0,1.000000,0.051343,0.0,...,0,0.0,0,0.0,0,0,0.000000,0,0,0.000000
2431,commu05506,1.000000,0.039367,0.211516,0.097267,0.040421,0.0,0.211516,0.044020,0.0,...,0,0.0,0,0.0,0,0,0.000000,0,0,0.000000
2432,commu05507,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0,0.0,0,0.0,0,0,0.418683,0,0,0.418683
2433,commu05508,0.040421,0.033475,0.047145,0.000000,1.000000,0.0,0.047145,0.196529,0.0,...,0,0.0,0,0.0,0,0,0.000000,0,0,0.000000


In [ ]:
slow_2.to_csv('slow_2.csv')

In [ ]:
for id_1 in tqdm(id_list_2) :

  for id_2 in id_list_2 :

    if id_1 < id_2 :

      A = slow_tf_idf_boc_n3.loc[slow_tf_idf_boc_n3['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = slow_tf_idf_boc_n3.loc[slow_tf_idf_boc_n3['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      slow_4.loc[slow_4['id'] == id_1 , id_2] = cos_sim(A , B)

slow_4

100%|██████████| 2435/2435 [1:19:50<00:00,  1.97s/it]


,id,commu05516,commu05517,commu05519,commu05520,commu05521,commu05523,commu05525,commu05526,commu05528,...,commu11114,commu11116,commu11117,commu11120,commu11124,commu11125,commu11128,commu11137,commu11138,commu11139
0,commu05516,0,0.039367,0.211516,0.097267,0.040421,0.000000,0.211516,0.044020,0,...,0,0.0,0,0.0,0,0,0.0,0,0,0.0
1,commu05517,0,0.000000,0.045916,0.000000,0.033475,0.000000,0.045916,0.036456,0,...,0,0.0,0,0.0,0,0,0.0,0,0,0.0
2,commu05519,0,0.000000,0.000000,0.000000,0.047145,0.000000,1.000000,0.051343,0,...,0,0.0,0,0.0,0,0,0.0,0,0,0.0
3,commu05520,0,0.000000,0.000000,0.000000,0.000000,0.076496,0.000000,0.385783,0,...,0,0.0,0,0.0,0,0,0.0,0,0,0.0
4,commu05521,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.047145,0.196529,0,...,0,0.0,0,0.0,0,0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430,commu11125,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0,0.0,0,0.0,0,0,0.0,1,1,0.0
2431,commu11128,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0,0.0,0,0.0,0,0,0.0,0,0,1.0
2432,commu11137,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0,0.0,0,0.0,0,0,0.0,0,1,0.0
2433,commu11138,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0,0.0,0,0.0,0,0,0.0,0,0,0.0


In [ ]:
slow_4.to_csv('slow_4.csv')

In [ ]:
slow_12 = pd.concat([slow_1 , slow_2] , axis = 1)
slow_34 = pd.concat([slow_3 , slow_4] , axis = 1)

slow_similarity_df_n3 = pd.concat([slow_12 , slow_34])

slow_similarity_df_n3

,id,commu00002,commu00023,commu00024,commu00036,commu00041,commu00042,commu00043,commu00046,commu00054,...,commu11114,commu11116,commu11117,commu11120,commu11124,commu11125,commu11128,commu11137,commu11138,commu11139
0,commu00002,0,0,1.0,0,0.118854,0,0.000000,0.248915,0,...,0,0.0,0,0.0,0,0,0.0,0,0,0.0
1,commu00023,0,0,0.0,0,0.000000,0,0.000000,0.000000,0,...,0,0.0,0,0.0,0,0,0.0,0,0,0.0
2,commu00024,0,0,0.0,0,0.118854,0,0.000000,0.248915,0,...,0,0.0,0,0.0,0,0,0.0,0,0,0.0
3,commu00036,0,0,0.0,0,0.000000,0,0.000000,0.000000,0,...,0,0.0,0,0.0,0,0,0.0,0,0,0.0
4,commu00041,0,0,0.0,0,0.000000,0,0.154447,0.000000,0,...,0,0.0,0,0.0,0,0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430,commu11125,0,0,0.0,0,0.000000,0,0.000000,0.000000,0,...,0,0.0,0,0.0,0,0,0.0,1,1,0.0
2431,commu11128,0,0,0.0,0,0.000000,0,0.000000,0.000000,0,...,0,0.0,0,0.0,0,0,0.0,0,0,1.0
2432,commu11137,0,0,0.0,0,0.000000,0,0.000000,0.000000,0,...,0,0.0,0,0.0,0,0,0.0,0,1,0.0
2433,commu11138,0,0,0.0,0,0.000000,0,0.000000,0.000000,0,...,0,0.0,0,0.0,0,0,0.0,0,0,0.0


In [ ]:
slow_similarity_df_n3.to_csv('cos_sim_slow_n3.csv')

### C) Fast (90 < bpm <= 120)

In [ ]:
fast_tf_idf_boc_n3 = pd.read_csv('TF-IDF_Bag_of_Chords_fast_n3.csv')

fast_tf_idf_boc_n3.drop('Unnamed: 0' , axis = 1 , inplace = True)

fast_tf_idf_boc_n3

,id,TF-IDF Bag of Chords
0,commu00001,"[0.3211830047270079, 0.7219794422336495, 0.554..."
1,commu00004,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.46537940..."
2,commu00006,"[0.3211830047270079, 0.7219794422336495, 0.0, ..."
3,commu00007,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,commu00008,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
2437,commu11133,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2438,commu11135,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2439,commu11136,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2440,commu11140,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
test = fast_tf_idf_boc_n3['TF-IDF Bag of Chords']

for i in range(len(test)) :

  test[i] = test[i].replace("'" , "").replace(" " , "").replace("[" , "").replace("]" , "")
  test[i] = test[i].split(",")

  test[i] = list(map(float , test[i]))

fast_tf_idf_boc_n3['TF-IDF Bag of Chords'] = test

In [ ]:
id_ord = list(fast_tf_idf_boc_n3['id'])

id_order = CategoricalDtype(categories = id_ord , ordered = True)

fast_tf_idf_boc_n3['id'] = fast_tf_idf_boc_n3['id'].astype(id_order)

fast_tf_idf_boc_n3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2442 entries, 0 to 2441
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    2442 non-null   category
 1   TF-IDF Bag of Chords  2442 non-null   object  
dtypes: category(1), object(1)
memory usage: 107.6+ KB


In [ ]:
id = fast_tf_idf_boc_n3['id']

id

0       commu00001
1       commu00004
2       commu00006
3       commu00007
4       commu00008
           ...    
2437    commu11133
2438    commu11135
2439    commu11136
2440    commu11140
2441    commu11141
Name: id, Length: 2442, dtype: category
Categories (2442, object): ['commu00001' < 'commu00004' < 'commu00006' < 'commu00007' < ... <
                            'commu11135' < 'commu11136' < 'commu11140' < 'commu11141']

In [ ]:
fast_similarity_df_n3 = pd.DataFrame(np.repeat(0 , len(id)**2).reshape(len(id),len(id)))

fast_similarity_df_n3

,0,1,2,3,4,5,6,7,8,9,...,2432,2433,2434,2435,2436,2437,2438,2439,2440,2441
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2439,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2440,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
id_list_1 = fast_tf_idf_boc_n3['id'][:1221]
id_list_2 = fast_tf_idf_boc_n3['id'][1221:]

In [ ]:
fast_1 = fast_similarity_df_n3.iloc[:1221 , :1221].copy()
fast_2 = fast_similarity_df_n3.iloc[:1221 , :1221].copy()
fast_3 = fast_similarity_df_n3.iloc[:1221 , :1221].copy()
fast_4 = fast_similarity_df_n3.iloc[:1221 , :1221].copy()

id_list_1 = list(fast_tf_idf_boc_n3['id'][:1221])
id_list_2 = list(fast_tf_idf_boc_n3['id'][1221:])

fast_1.columns = id_list_1
fast_1.insert(0 , 'id' , id_list_1)

fast_2.columns = id_list_2
fast_2.insert(0 , 'id' , id_list_1)

fast_3.columns = id_list_1
fast_3.insert(0 , 'id' , id_list_2)

fast_4.columns = id_list_2
fast_4.insert(0 , 'id' , id_list_2)

In [ ]:
for id_1 in tqdm(id_list_1) :

  for id_2 in id_list_1 :

    if id_1 < id_2 :

      A = fast_tf_idf_boc_n3.loc[fast_tf_idf_boc_n3['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = fast_tf_idf_boc_n3.loc[fast_tf_idf_boc_n3['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      fast_1.loc[fast_1['id'] == id_1 , id_2] = cos_sim(A , B)

fast_1

100%|██████████| 1221/1221 [18:47<00:00,  1.08it/s]


,id,commu00001,commu00004,commu00006,commu00007,commu00008,commu00009,commu00016,commu00017,commu00018,...,commu03332,commu03334,commu03342,commu03344,commu03349,commu03353,commu03360,commu03361,commu03363,commu03367
0,commu00001,0,0,0.598957,0,0,0.000000,0,0.0,0,...,0.0,0.336671,0.0,0.0,0.0,0.336671,0.0,0.0,0.0,0.0
1,commu00004,0,0,0.000000,0,0,0.000000,0,1.0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,commu00006,0,0,0.000000,0,0,0.000000,0,0.0,0,...,0.0,0.314722,0.0,0.0,0.0,0.314722,0.0,0.0,0.0,0.0
3,commu00007,0,0,0.000000,0,0,0.000000,0,0.0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,commu00008,0,0,0.000000,0,0,0.639046,0,0.0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,commu03353,0,0,0.000000,0,0,0.000000,0,0.0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1217,commu03360,0,0,0.000000,0,0,0.000000,0,0.0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0
1218,commu03361,0,0,0.000000,0,0,0.000000,0,0.0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0
1219,commu03363,0,0,0.000000,0,0,0.000000,0,0.0,0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [ ]:
for id_1 in tqdm(id_list_1) :

  for id_2 in id_list_2 :

    if id_1 < id_2 :

      A = fast_tf_idf_boc_n3.loc[fast_tf_idf_boc_n3['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = fast_tf_idf_boc_n3.loc[fast_tf_idf_boc_n3['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      fast_2.loc[fast_2['id'] == id_1 , id_2] = cos_sim(A , B)

fast_2

100%|██████████| 1221/1221 [38:03<00:00,  1.87s/it]


,id,commu03368,commu03370,commu03375,commu03387,commu03399,commu03433,commu03435,commu03436,commu03445,...,commu11126,commu11129,commu11130,commu11131,commu11132,commu11133,commu11135,commu11136,commu11140,commu11141
0,commu00001,0.336671,0.000000,0.089963,0.0,0.0,0.0,0,0.336671,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.000000
1,commu00004,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.000000,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.000000
2,commu00006,0.314722,0.000000,0.191016,0.0,0.0,0.0,0,0.314722,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.000000
3,commu00007,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.000000,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.000000
4,commu00008,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.000000,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,commu03353,1.000000,0.000000,0.000000,0.0,0.0,0.0,0,1.000000,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.000000
1217,commu03360,0.000000,0.919215,0.000000,1.0,0.0,0.0,0,0.000000,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.104321
1218,commu03361,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.000000,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.000000
1219,commu03363,0.000000,0.919215,0.000000,1.0,0.0,0.0,0,0.000000,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.104321


In [ ]:
for id_1 in tqdm(id_list_2) :

  for id_2 in id_list_2 :

    if id_1 < id_2 :

      A = fast_tf_idf_boc_n3.loc[fast_tf_idf_boc_n3['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = fast_tf_idf_boc_n3.loc[fast_tf_idf_boc_n3['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      fast_4.loc[fast_4['id'] == id_1 , id_2] = cos_sim(A , B)

fast_4

100%|██████████| 1221/1221 [18:59<00:00,  1.07it/s]


,id,commu03368,commu03370,commu03375,commu03387,commu03399,commu03433,commu03435,commu03436,commu03445,...,commu11126,commu11129,commu11130,commu11131,commu11132,commu11133,commu11135,commu11136,commu11140,commu11141
0,commu03368,0,0,0,0.000000,0.00000,0.00000,0,1,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
1,commu03370,0,0,0,0.919215,0.00000,0.00000,0,0,0.00000,...,0.0,0.0,0.0,0.0,0.101902,0.0,0.0,0.0,0.000000,0.053097
2,commu03375,0,0,0,0.000000,0.27515,0.27515,0,0,0.27515,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
3,commu03387,0,0,0,0.000000,0.00000,0.00000,0,0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.104321
4,commu03399,0,0,0,0.000000,0.00000,1.00000,0,0,1.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,commu11133,0,0,0,0.000000,0.00000,0.00000,0,0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
1217,commu11135,0,0,0,0.000000,0.00000,0.00000,0,0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.181486,0.000000
1218,commu11136,0,0,0,0.000000,0.00000,0.00000,0,0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.181486,0.000000
1219,commu11140,0,0,0,0.000000,0.00000,0.00000,0,0,0.00000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000


In [ ]:
fast_12 = pd.concat([fast_1 , fast_2] , axis = 1)
fast_34 = pd.concat([fast_3 , fast_4] , axis = 1)

fast_similarity_df_n3 = pd.concat([fast_12 , fast_34])

fast_similarity_df_n3

,id,commu00001,commu00004,commu00006,commu00007,commu00008,commu00009,commu00016,commu00017,commu00018,...,commu11126,commu11129,commu11130,commu11131,commu11132,commu11133,commu11135,commu11136,commu11140,commu11141
0,commu00001,0,0,0.598957,0,0,0.000000,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,commu00004,0,0,0.000000,0,0,0.000000,0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,commu00006,0,0,0.000000,0,0,0.000000,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,commu00007,0,0,0.000000,0,0,0.000000,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,commu00008,0,0,0.000000,0,0,0.639046,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,commu11133,0,0,0.000000,0,0,0.000000,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1217,commu11135,0,0,0.000000,0,0,0.000000,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.181486,0.0
1218,commu11136,0,0,0.000000,0,0,0.000000,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.181486,0.0
1219,commu11140,0,0,0.000000,0,0,0.000000,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [ ]:
fast_similarity_df_n3.to_csv('cos_sim_fast_n3.csv')

### D) Very_fast (120 < bpm)

In [ ]:
very_fast_tf_idf_boc_n3 = pd.read_csv('TF-IDF_Bag_of_Chords_very_fast_n3.csv')

very_fast_tf_idf_boc_n3.drop('Unnamed: 0' , axis = 1 , inplace = True)

very_fast_tf_idf_boc_n3

,id,TF-IDF Bag of Chords
0,commu00003,"[0.5689861771095454, 0.8167841871213636, 0.816..."
1,commu00010,"[0.5689861771095454, 0.8167841871213636, 0.816..."
2,commu00012,"[0.0, 0.0, 0.0, 0.0, 0.838537031368771, 0.8385..."
3,commu00013,"[0.5689861771095454, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,commu00019,"[0.5689861771095454, 0.8167841871213636, 0.816..."
...,...,...
1884,commu11104,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1885,commu11127,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1886,commu11142,"[0.5689861771095454, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1887,commu11143,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
test = very_fast_tf_idf_boc_n3['TF-IDF Bag of Chords']

for i in range(len(test)) :

  test[i] = test[i].replace("'" , "").replace(" " , "").replace("[" , "").replace("]" , "")
  test[i] = test[i].split(",")

  test[i] = list(map(float , test[i]))

very_fast_tf_idf_boc_n3['TF-IDF Bag of Chords'] = test

In [ ]:
id_ord = list(very_fast_tf_idf_boc_n3['id'])

id_order = CategoricalDtype(categories = id_ord , ordered = True)

very_fast_tf_idf_boc_n3['id'] = very_fast_tf_idf_boc_n3['id'].astype(id_order)

very_fast_tf_idf_boc_n3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    1889 non-null   category
 1   TF-IDF Bag of Chords  1889 non-null   object  
dtypes: category(1), object(1)
memory usage: 97.9+ KB


In [ ]:
very_fast_tf_idf_boc_n3['id']

0       commu00003
1       commu00010
2       commu00012
3       commu00013
4       commu00019
           ...    
1884    commu11104
1885    commu11127
1886    commu11142
1887    commu11143
1888    commu11144
Name: id, Length: 1889, dtype: category
Categories (1889, object): ['commu00003' < 'commu00010' < 'commu00012' < 'commu00013' < ... <
                            'commu11127' < 'commu11142' < 'commu11143' < 'commu11144']

In [ ]:
id = very_fast_tf_idf_boc_n3['id']

id

0       commu00003
1       commu00010
2       commu00012
3       commu00013
4       commu00019
           ...    
1884    commu11104
1885    commu11127
1886    commu11142
1887    commu11143
1888    commu11144
Name: id, Length: 1889, dtype: category
Categories (1889, object): ['commu00003' < 'commu00010' < 'commu00012' < 'commu00013' < ... <
                            'commu11127' < 'commu11142' < 'commu11143' < 'commu11144']

In [ ]:
very_fast_similarity_df_n3 = pd.DataFrame(np.repeat(0 , len(id)**2).reshape(len(id),len(id)))

very_fast_similarity_df_n3

,0,1,2,3,4,5,6,7,8,9,...,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1885,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
id_list_1 = very_fast_tf_idf_boc_n3['id'][:945]
id_list_2 = very_fast_tf_idf_boc_n3['id'][945:]

In [ ]:
very_fast_1 = very_fast_similarity_df_n3.iloc[:945 , :945].copy()
very_fast_2 = very_fast_similarity_df_n3.iloc[:945 , 945:].copy()
very_fast_3 = very_fast_similarity_df_n3.iloc[945: , :945].copy()
very_fast_4 = very_fast_similarity_df_n3.iloc[945: , 945:].copy()

id_list_1 = list(very_fast_tf_idf_boc_n3['id'][:945])
id_list_2 = list(very_fast_tf_idf_boc_n3['id'][945:])

very_fast_1.columns = id_list_1
very_fast_1.insert(0 , 'id' , id_list_1)

very_fast_2.columns = id_list_2
very_fast_2.insert(0 , 'id' , id_list_1)

very_fast_3.columns = id_list_1
very_fast_3.insert(0 , 'id' , id_list_2)

very_fast_4.columns = id_list_2
very_fast_4.insert(0 , 'id' , id_list_2)

In [ ]:
for id_1 in tqdm(id_list_1) :

  for id_2 in id_list_1 :

    if id_1 < id_2 :

      A = very_fast_tf_idf_boc_n3.loc[very_fast_tf_idf_boc_n3['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = very_fast_tf_idf_boc_n3.loc[very_fast_tf_idf_boc_n3['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      very_fast_1.loc[very_fast_1['id'] == id_1 , id_2] = cos_sim(A , B)

very_fast_1

100%|██████████| 945/945 [11:51<00:00,  1.33it/s]


,id,commu00003,commu00010,commu00012,commu00013,commu00019,commu00021,commu00027,commu00047,commu00052,...,commu07124,commu07135,commu07138,commu07142,commu07152,commu07158,commu07159,commu07162,commu07164,commu07178
0,commu00003,0,1,0,0.157812,1.000000,0.00000,0,0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,commu00010,0,0,0,0.157812,1.000000,0.00000,0,0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,commu00012,0,0,0,0.000000,0.000000,0.00000,1,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,commu00013,0,0,0,0.000000,0.157812,0.14748,0,0,0.157812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,commu00019,0,0,0,0.000000,0.000000,0.00000,0,0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,commu07158,0,0,0,0.000000,0.000000,0.00000,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,1.0
941,commu07159,0,0,0,0.000000,0.000000,0.00000,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066349,0.0,0.0
942,commu07162,0,0,0,0.000000,0.000000,0.00000,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
943,commu07164,0,0,0,0.000000,0.000000,0.00000,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0


In [ ]:
for id_1 in tqdm(id_list_1) :

  for id_2 in id_list_2 :

    if id_1 < id_2 :

      A = very_fast_tf_idf_boc_n3.loc[very_fast_tf_idf_boc_n3['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = very_fast_tf_idf_boc_n3.loc[very_fast_tf_idf_boc_n3['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      very_fast_2.loc[very_fast_2['id'] == id_1 , id_2] = cos_sim(A , B)

very_fast_2

100%|██████████| 945/945 [23:03<00:00,  1.46s/it]


,id,commu07185,commu07186,commu07193,commu07199,commu07201,commu07206,commu07209,commu07223,commu07253,...,commu11091,commu11093,commu11094,commu11098,commu11099,commu11104,commu11127,commu11142,commu11143,commu11144
0,commu00003,0.000000,0,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0,0.000000,0.000000,0.000000,0.100261,0,0
1,commu00010,0.000000,0,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0,0.000000,0.000000,0.000000,0.100261,0,0
2,commu00012,0.000000,0,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0,0
3,commu00013,0.000000,0,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0,0.000000,0.000000,0.000000,0.113637,0,0
4,commu00019,0.000000,0,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0,0.000000,0.000000,0.000000,0.100261,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,commu07158,0.622905,0,1.0,1.0,1.0,0.000000,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0,0
941,commu07159,0.000000,0,0.0,0.0,0.0,0.000000,0.05531,0.05531,0.0,...,0.05531,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0,0
942,commu07162,0.000000,0,0.0,0.0,0.0,0.162284,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0,0.279139,0.279139,0.166737,0.000000,0,0
943,commu07164,0.622905,0,1.0,1.0,1.0,0.000000,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000,0,0


In [ ]:
for id_1 in tqdm(id_list_2) :

  for id_2 in id_list_2 :

    if id_1 < id_2 :

      A = very_fast_tf_idf_boc_n3.loc[very_fast_tf_idf_boc_n3['id'] == id_1 , 'TF-IDF Bag of Chords'].iloc[0]
      B = very_fast_tf_idf_boc_n3.loc[very_fast_tf_idf_boc_n3['id'] == id_2 , 'TF-IDF Bag of Chords'].iloc[0]

      very_fast_4.loc[very_fast_4['id'] == id_1 , id_2] = cos_sim(A , B)

very_fast_4

100%|██████████| 944/944 [11:28<00:00,  1.37it/s]


,id,commu07185,commu07186,commu07193,commu07199,commu07201,commu07206,commu07209,commu07223,commu07253,...,commu11091,commu11093,commu11094,commu11098,commu11099,commu11104,commu11127,commu11142,commu11143,commu11144
945,commu07185,0,0,0.622905,0.622905,0.622905,0,0.053703,0.053703,0,...,0.053703,0.0,0.079142,0.0,0.0,0.0,0.000000,0.0,0,0.0
946,commu07186,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0.0
947,commu07193,0,0,0.000000,1.000000,1.000000,0,0.000000,0.000000,0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0.0
948,commu07199,0,0,0.000000,0.000000,1.000000,0,0.000000,0.000000,0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0.0
949,commu07201,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,commu11104,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.403594,0.0,0,0.0
1885,commu11127,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0.0
1886,commu11142,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0.0
1887,commu11143,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0.0


In [ ]:
very_fast_12 = pd.concat([very_fast_1 , very_fast_2] , axis = 1)
very_fast_34 = pd.concat([very_fast_3 , very_fast_4] , axis = 1)

very_fast_similarity_df_n3 = pd.concat([very_fast_12 , very_fast_34])

very_fast_similarity_df_n3

,id,commu00003,commu00010,commu00012,commu00013,commu00019,commu00021,commu00027,commu00047,commu00052,...,commu11091,commu11093,commu11094,commu11098,commu11099,commu11104,commu11127,commu11142,commu11143,commu11144
0,commu00003,0,1,0,0.157812,1.000000,0.00000,0,0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.100261,0,0.0
1,commu00010,0,0,0,0.157812,1.000000,0.00000,0,0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.100261,0,0.0
2,commu00012,0,0,0,0.000000,0.000000,0.00000,1,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0.0
3,commu00013,0,0,0,0.000000,0.157812,0.14748,0,0,0.157812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.113637,0,0.0
4,commu00019,0,0,0,0.000000,0.000000,0.00000,0,0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.100261,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,commu11104,0,0,0,0.000000,0.000000,0.00000,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.403594,0.000000,0,0.0
1885,commu11127,0,0,0,0.000000,0.000000,0.00000,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0.0
1886,commu11142,0,0,0,0.000000,0.000000,0.00000,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0.0
1887,commu11143,0,0,0,0.000000,0.000000,0.00000,0,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0.0


In [ ]:
very_fast_similarity_df_n3.to_csv('cos_sim_very_fast_n3.csv')